In [1]:
# Modified Primal Tableau generic format is:

# con\var |    b        x1        x2        ...        xn        s1        s2        ...        sq        A1        A2        ...        Ar
# ---------------------------------------------------------------------------------------------------------------------------------------------
# con1    |    b1       x11       x11       ...        x1n       0/1/-1    0/1/-1    ...        0/1/-1    0/1       0/1       ...        0/1
# con2    |    b2       x21       x22       ...        x2n       0/1/-1    0/1/-1    ...        0/1/-1    0/1       0/1       ...        0/1
# .       |    .        .         .         ...        .         .         .         ...        .         .         .         ...        .
# .       |    .        .         .         ...        .         .         .         ...        .         .         .         ...        .
# .       |    .        .         .         ...        .         .         .         ...        .         .         .         ...        .
# conm    |    bm       xm1       xm2       ...        xmn       0/1/-1    0/1/-1    ...        0/1/-1    0/1       0/1       ...        0/1
# z       |    z0       -c1       -c2       ...        0         0         0         ...        0         0         0         ...        0
# zA      |    0        0         0         ...        0         0         0         ...        0         -1        -1        ...        -1

In [2]:
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()
print(tf.__version__)

1.12.0


In [3]:
tf.executing_eagerly()

False

In [4]:
max_branch_and_bound_recursion_depth = tf.constant(value = 6, dtype = tf.int64)

In [5]:
# Objective function
# z = 1 * x1 + -1 * x2

In [6]:
# Constraints
# -2 * x1 + 2 * x2 >= 1
# 1 * x1 + 1 * x2 <= 10
# -8 * x1 + 10 * x2 >= 13
# 1 * x1 + 1 * x2 <= 9

# Add slack/surplus variables
# -2 * x1 + 2 * x2 - s1 = 1
# 1 * x1 + 1 * x2 + s2 = 10
# -8 * x1 + 10 * x2 - s3 = 13
# 1 * x1 + 1 * x2 + s4 = 9

# Converts to
# 1 = -2 * x1 + 2 * x2 - s1
# 10 = 1 * x1 + 1 * x2 + s2
# 13 = -8 * x1 + 10 * x2 - s3
# 9 = 1 * x1 + 1 * x2 + s4

In [7]:
# Constraints in tableau format are
# con\var |    b        x1        x2        s1        s2        s3        s4        A1        A2
# -----------------------------------------------------------------------------------------------
# con1    |    1        -2        2         1         0         0         0         1         0
# con2    |    10       1         1         0         1         0         0         0         0
# con3    |    13       -8        10        0         0         1         0         0         1
# con4    |    9        1         1         0         0         0         1         0         0
# z       |    0        -1        1         0         0         0         0         0         0
# zA      |    0        0         0         0         0         0         0         -1        -1

# Read inputs

### Define if this is a maximization or minimization problem

In [8]:
# This function reads in if the objective function is going to be a maximization or minimization problem
def read_objective_function_maximization_or_minimization():
    maximization_problem = np.loadtxt(fname = "inputs/objective_function_maximization.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_objective_function_maximization_or_minimization: maximization_problem = \n{}".format(maximization_problem))
    
    return maximization_problem

### Get number of constraints and variables

In [9]:
# This function reads the number of constraints and variables
def read_number_of_constraints_and_variables():
    number_of_constraints = np.loadtxt(fname = "inputs/number_of_constraints.txt", dtype = np.int64, delimiter = '\t')
    number_of_variables = np.loadtxt(fname = "inputs/number_of_variables.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_number_of_constraints_and_variables: number_of_constraints = \n{}".format(number_of_constraints))
    print("read_number_of_constraints_and_variables: number_of_variables = \n{}".format(number_of_variables))
    
    return number_of_constraints, number_of_variables

### Get variable special requirements

In [10]:
# This function reads and counts variable special requirements like needing to be an integer, etc.
def read_and_count_variable_special_requirements():
    # shape = (number_of_variables,)
    variable_special_requirements = np.loadtxt(fname = "inputs/variable_special_requirements.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_and_count_variable_special_requirements: variable_special_requirements = \n{}".format(variable_special_requirements))

    number_of_variables_required_to_be_standard = np.count_nonzero(a = (variable_special_requirements == 0))
    number_of_variables_required_to_be_integer = np.count_nonzero(a = (variable_special_requirements == 1))
    number_of_variables_required_to_be_binary = np.count_nonzero(a = (variable_special_requirements == 2))
    number_of_variables_required_to_be_unrestricted = np.count_nonzero(a = (variable_special_requirements == 3))

    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_standard = \n{}".format(number_of_variables_required_to_be_standard))
    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_integer = \n{}".format(number_of_variables_required_to_be_integer))
    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_binary = \n{}".format(number_of_variables_required_to_be_binary))
    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_unrestricted = \n{}".format(number_of_variables_required_to_be_unrestricted))
    
    return variable_special_requirements, number_of_variables_required_to_be_standard, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, number_of_variables_required_to_be_unrestricted

### Create objective function

In [11]:
# This function reads in the objective function's initial constant
def read_objective_function_initial_constant():
    # shape = (2,)
    objective_function_initial_constant_numerator = np.loadtxt(fname = "inputs/objective_function_initial_constant_numerator.txt", dtype = np.int64, delimiter = '\t')
    objective_function_initial_constant_denominator = np.loadtxt(fname = "inputs/objective_function_initial_constant_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    objective_function_initial_constant = np.stack(arrays = [objective_function_initial_constant_numerator, objective_function_initial_constant_denominator], axis = 0)
    
    print("read_objective_function_initial_constant: objective_function_initial_constant = \n{}".format(objective_function_initial_constant[0].astype(np.float64) / objective_function_initial_constant[1]))
    
    return objective_function_initial_constant

In [12]:
# This function reads the objective functions variable coefficients
def read_objective_function_coefficient_vector():
    # shape = (2, number_of_variables)
    objective_function_coefficient_vector_numerator = np.loadtxt(fname = "inputs/objective_function_coefficient_vector_numerator.txt", dtype = np.int64, delimiter = '\t')
    objective_function_coefficient_vector_denominator = np.loadtxt(fname = "inputs/objective_function_coefficient_vector_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    objective_function_coefficient_vector = np.stack(arrays = [objective_function_coefficient_vector_numerator, objective_function_coefficient_vector_denominator], axis = 0)
    
    print("read_objective_function_coefficient_vector: objective_function_coefficient_vector = \n{}".format(objective_function_coefficient_vector[0].astype(np.float64) / objective_function_coefficient_vector[1]))
    
    return objective_function_coefficient_vector

### Get initial constraint inequality directions (<= (1), >= (-1), = (0))

In [13]:
# This function reads the constraint inequality directions
def read_constraint_inequality_direction_vector():
    # shape = (number_of_constraints,)
    constraint_inequality_direction_vector = np.loadtxt(fname = "inputs/constraint_inequality_direction_vector.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_constraint_inequality_direction_vector: constraint_inequality_direction_vector = \n{}".format(constraint_inequality_direction_vector))
    
    return constraint_inequality_direction_vector

### Get constraint constants

In [14]:
# This function reads the constraint constants
def read_constraint_constant_vector():
    # shape = (number_of_constraints,)
    constraint_constant_vector_numerator = np.loadtxt(fname = "inputs/constraint_constant_vector_numerator.txt", dtype = np.int64, delimiter = '\t')
    constraint_constant_vector_denominator = np.loadtxt(fname = "inputs/constraint_constant_vector_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    constraint_constant_vector = np.stack(arrays = [constraint_constant_vector_numerator, constraint_constant_vector_denominator], axis = 0)
    
    print("read_constraint_constant_vector: constraint_constant_vector = \n{}".format(constraint_constant_vector[0].astype(np.float64) / constraint_constant_vector[1]))
    
    return constraint_constant_vector

### Get constraint coefficient matrix

In [15]:
# This function reads in the constraint coefficient matrix
def read_constraint_coefficient_matrix():
    # shape = (2, number_of_constraints, number_of_variables)
    constraint_coefficient_matrix_numerator = np.loadtxt(fname = "inputs/constraint_coefficient_matrix_numerator.txt", dtype = np.int64, delimiter = '\t')
    constraint_coefficient_matrix_denominator = np.loadtxt(fname = "inputs/constraint_coefficient_matrix_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    constraint_coefficient_matrix = np.stack(arrays = [constraint_coefficient_matrix_numerator, constraint_coefficient_matrix_denominator], axis = 0)
    
    print("read_constraint_coefficient_matrix: constraint_coefficient_matrix = \n{}".format(constraint_coefficient_matrix[0].astype(np.float64) / constraint_coefficient_matrix[1]))
    
    return constraint_coefficient_matrix

# Optimal values

### Define placeholders for the optimal parameters

In [16]:
# This function creates the variables to hold the optimal opjective function value and associated variable values
def create_optimal_objective_function_and_variable_values(maximization_problem, number_of_variables):
    # shape = (2,)
    optimal_objective_function_value = tf.stack(values = [tf.where(condition = maximization_problem == 1, x = np.iinfo(np.int64).min, y = np.iinfo(np.int64).max), tf.ones(shape = (), dtype = tf.int64)], axis = 0)

    # shape = (2, number_of_variables)
    optimal_variable_values = tf.stack(values = [tf.zeros(shape = [number_of_variables], dtype = tf.int64), tf.ones(shape = [number_of_variables], dtype = tf.int64)], axis = 0)
    
    return optimal_objective_function_value, optimal_variable_values

# Mixed Integer Linear Programming

In [17]:
# This function performs mixed integer linear programming
def mixed_integer_linear_programming(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix, optimal_objective_function_value, optimal_variable_values):
    error_code = tf.constant(value = 0, dtype = tf.int64)

    print("\n\n************************************************************************************************************\n")
    print("************************************************************************************************************\n")
    print("*************************************************** MILP ***************************************************\n")
    print("************************************************************************************************************\n")
    print("************************************************************************************************************\n\n\n")

    # modify_constraint_inequality_directions_and_count_directions
    number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, modified_constraint_inequality_direction_vector = \
        modify_constraint_inequality_directions_and_count_directions(number_of_constraints, number_of_variables_required_to_be_binary, constraint_constant_vector, constraint_inequality_direction_vector)

    number_of_slack_surplus_variables = number_of_less_than_or_equal_to_constraints + number_of_greater_than_or_equal_to_constraints
    number_of_artificial_variables = number_of_equal_to_constraints + number_of_greater_than_or_equal_to_constraints
    
    # initialize_current_tableau_row_and_column_counts
    tableau_current_size = initialize_current_tableau_row_and_column_counts(number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables)

    # create_tableau_matrix
    number_of_constraints, tableau_matrix = create_tableau_matrix(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, modified_constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix)

#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    # create_basic_variables
    basic_variables = create_basic_variables(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix)

    # create_basic_feasible_solution
    basic_feasible_solution = create_basic_feasible_solution(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, tableau_matrix)
    
    # write_initial_LP_values
    write_initial_LP_values(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix)

    # print_initial_counts
    print_initial_counts(number_of_constraints, number_of_variables, number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size)

#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    # Solve LP Relaxation

    # This function finds the optimal solution for the given variables and constraints
    # simplex_algorithm
    error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = simplex_algorithm(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size)

    condition_optimal_lp_relaxation = tf.equal(x = error_code, y = 0)
    print("mixed_integer_linear_programming: condition_optimal_lp_relaxation = \n{}".format(condition_optimal_lp_relaxation))
    condition_infeasible_lp_relaxation = tf.equal(x = error_code, y = 1)
    print("mixed_integer_linear_programming: condition_infeasible_lp_relaxation = \n{}".format(condition_infeasible_lp_relaxation))
    condition_unbounded_lp_relaxation = tf.equal(x = error_code, y = 2)
    print("mixed_integer_linear_programming: condition_unbounded_lp_relaxation = \n{}".format(condition_unbounded_lp_relaxation))
    condition_unknown_lp_relaxation_error = tf.logical_not(x = tf.reduce_any(input_tensor = [condition_optimal_lp_relaxation, condition_infeasible_lp_relaxation, condition_unbounded_lp_relaxation]))
    print("mixed_integer_linear_programming: condition_unknown_lp_relaxation_error = \n{}".format(condition_unknown_lp_relaxation_error))
    
    # If LP is optimal then MILP will be either infeasible or optimal
    def optimal_lp_relaxation(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code):
        print("mixed_integer_linear_programming: optimal_lp_relaxation: LP is optimal!")

        # update_optimal_objective_function_and_variable_solution
        optimal_objective_function_value, optimal_variable_values = update_optimal_objective_function_and_variable_solution(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix)

        # print_optimal_results
        print_optimal_results(optimal_objective_function_value, optimal_variable_values)
        
        condition_optimal_still_need_integer_or_binary_variables = tf.greater(x = number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary, y = 0)
        print("mixed_integer_linear_programming: optimal_lp_relaxation: condition_optimal_still_need_integer_or_binary_variables = \n{}".format(condition_optimal_still_need_integer_or_binary_variables))
        
        def optimal_still_need_integer_or_binary_variables(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code):
            print("\n\n************************************************************************************************************\n")
            print("************************************************************************************************************\n")
            print("*********************************************  BRANCH AND BOUND ********************************************\n")
            print("************************************************************************************************************\n")
            print("************************************************************************************************************\n\n\n")

            # branch_and_bound_MILP
            error_code, optimal_objective_function_value, optimal_variable_values = branch_and_bound_MILP(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code)

#             if error_code == 0:
#                 print("mixed_integer_linear_programming: LP was optimal, MILP is optimal!")

#                 write_final_MILP_values(optimal_objective_function_value, optimal_variable_values)
#             elif error_code == 1:
#                 print("mixed_integer_linear_programming: LP was optimal, MILP is infeasible!")
#             elif error_code == 2:
#                 print("mixed_integer_linear_programming: LP was optimal, MILP is unbounded!  This is IMPOSSIBLE since LP was optimal!")
#             else:
#                 print("mixed_integer_linear_programming: WEIRD ERROR_CODE = {}".format(error_code))
            
            return error_code, optimal_objective_function_value, optimal_variable_values
        
        def optimal_dont_still_need_integer_or_binary_variables(error_code, optimal_objective_function_value, optimal_variable_values):
            return error_code, optimal_objective_function_value, optimal_variable_values

        error_code, optimal_objective_function_value, optimal_variable_values = tf.cond(
            pred = condition_optimal_still_need_integer_or_binary_variables, 
            true_fn = lambda: optimal_still_need_integer_or_binary_variables(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code), 
            false_fn = lambda: optimal_dont_still_need_integer_or_binary_variables(error_code, optimal_objective_function_value, optimal_variable_values))
                
        return error_code, optimal_objective_function_value, optimal_variable_values
    
    # If LP is infeasible then MILP will be infeasible since it is a subset of a NULL set
    def infeasible_lp_relaxation(error_code, optimal_objective_function_value, optimal_variable_values):
        print("mixed_integer_linear_programming: infeasible_lp_relaxation: LP is infeasible!")
        
        condition_infeasible_still_need_integer_or_binary_variables = tf.greater(x = number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary, y = 0)
        print("mixed_integer_linear_programming: infeasible_lp_relaxation: condition_infeasible_still_need_integer_or_binary_variables = \n{}".format(condition_infeasible_still_need_integer_or_binary_variables))
        
        def infeasible_still_need_integer_or_binary_variables():
            print("mixed_integer_linear_programming: infeasible_lp_relaxation: infeasible_still_need_integer_or_binary_variables: An infeasible LP ALWAYS leads to an infeasible MILP!")
            
            return tf.constant(value = 0, dtype = tf.int64)
        
        def infeasible_dont_still_need_integer_or_binary_variables():
            return tf.constant(value = 0, dtype = tf.int64)

        _ = tf.cond(
            pred = condition_infeasible_still_need_integer_or_binary_variables, 
            true_fn = lambda: infeasible_still_need_integer_or_binary_variables(), 
            false_fn = lambda: infeasible_dont_still_need_integer_or_binary_variables())
            
        return error_code, optimal_objective_function_value, optimal_variable_values
    
    # If LP is unbounded and all LP coefficients are rational then the MILP will either be infeasible or unbounded, however if LP has some irrational coefficients then MILP might be optimal instead
    def unbounded_lp_relaxation(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code):
        print("mixed_integer_linear_programming: unbounded_lp_relaxation: LP is unbounded!")
        
        condition_unbounded_still_need_integer_or_binary_variables = tf.greater(x = number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary, y = 0)
        print("mixed_integer_linear_programming: unbounded_lp_relaxation: condition_unbounded_still_need_integer_or_binary_variables = \n{}".format(condition_unbounded_still_need_integer_or_binary_variables))
        
        def unbounded_still_need_integer_or_binary_variables(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code):
            print("\n\n************************************************************************************************************\n")
            print("************************************************************************************************************\n")
            print("*********************************************  BRANCH AND BOUND ********************************************\n")
            print("************************************************************************************************************\n")
            print("************************************************************************************************************\n\n\n")

            error_code, optimal_objective_function_value, optimal_variable_values = branch_and_bound_MILP(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code)

#             if error_code == 0:
#                 print("mixed_integer_linear_programming: LP was unbounded, MILP is optimal!  There must be some irrational coefficients and/or binary variables!")
#             elif error_code == 1:
#                 print("mixed_integer_linear_programming: LP was unbounded, MILP is infeasible!")
#             elif error_code == 2:
#                 print("mixed_integer_linear_programming: LP was unbounded, MILP is unbounded!")
#             else:
#                 print("mixed_integer_linear_programming: WEIRD ERROR_CODE = {}".format(error_code))
                
            return error_code, optimal_objective_function_value, optimal_variable_values
        
        def unbounded_dont_still_need_integer_or_binary_variables(error_code, optimal_objective_function_value, optimal_variable_values):
            return error_code, optimal_objective_function_value, optimal_variable_values

        error_code, optimal_objective_function_value, optimal_variable_values = tf.cond(
            pred = condition_unbounded_still_need_integer_or_binary_variables, 
            true_fn = lambda: unbounded_still_need_integer_or_binary_variables(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code), 
            false_fn = lambda: unbounded_dont_still_need_integer_or_binary_variables(error_code, optimal_objective_function_value, optimal_variable_values))
        
        return error_code, optimal_objective_function_value, optimal_variable_values
    
    def unknown_lp_relaxation_error(error_code, optimal_objective_function_value, optimal_variable_values):
        print("mixed_integer_linear_programming: unknown_lp_relaxation_error: WEIRD ERROR_CODE = {}".format(error_code))        
        
        return error_code, optimal_objective_function_value, optimal_variable_values
    
    def lp_relaxation_do_nothing(error_code, optimal_objective_function_value, optimal_variable_values):
        return error_code, optimal_objective_function_value, optimal_variable_values

    unknown_lp_relaxation_error_branch = tf.cond(
        pred = condition_unbounded_lp_relaxation, 
        true_fn = lambda: unknown_lp_relaxation_error(error_code, optimal_objective_function_value, optimal_variable_values), 
        false_fn = lambda: lp_relaxation_do_nothing(error_code, optimal_objective_function_value, optimal_variable_values))
    
    unbounded_lp_relaxation_branch = tf.cond(
        pred = condition_unbounded_lp_relaxation, 
        true_fn = lambda: unbounded_lp_relaxation(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code), 
        false_fn = lambda: unknown_lp_relaxation_error_branch)
    
    infeasible_lp_relaxation_branch = tf.cond(
        pred = condition_infeasible_lp_relaxation, 
        true_fn = lambda: infeasible_lp_relaxation(error_code, optimal_objective_function_value, optimal_variable_values), 
        false_fn = lambda: unbounded_lp_relaxation_branch)
    
    optimal_lp_relaxation_branch = tf.cond(
        pred = condition_optimal_lp_relaxation, 
        true_fn = lambda: optimal_lp_relaxation(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code), 
        false_fn = lambda: infeasible_lp_relaxation_branch)
    
    error_code, optimal_objective_function_value, optimal_variable_values = optimal_lp_relaxation_branch
    
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    return error_code, optimal_objective_function_value, optimal_variable_values

In [18]:
# This function modifies the constraint inequality directions and then counts instances of each direction type
def modify_constraint_inequality_directions_and_count_directions(number_of_constraints, number_of_variables_required_to_be_binary, constraint_constant_vector, constraint_inequality_direction_vector):
    number_of_less_than_or_equal_to_constraints = number_of_variables_required_to_be_binary
    number_of_equal_to_constraints = tf.constant(value = 0, dtype = tf.int64)
    number_of_greater_than_or_equal_to_constraints = tf.constant(0, dtype = tf.int64)
    
    modified_constraint_inequality_direction_vector = constraint_inequality_direction_vector
    
    greater_than_or_equal_to_mask = tf.equal(x = constraint_inequality_direction_vector[0:number_of_constraints], y = -1)
    equal_to_mask = tf.equal(x = constraint_inequality_direction_vector[0:number_of_constraints], y = 0)
    less_than_or_equal_to_mask = tf.equal(x = constraint_inequality_direction_vector[0:number_of_constraints], y = 1)
    
    negative_constraint_constant_mask = tf.less(x = constraint_constant_vector[0, 0:number_of_constraints], y = 0)
    
    # if originally greater than or equal to
    condition = tf.reduce_all(input_tensor = [greater_than_or_equal_to_mask, negative_constraint_constant_mask], axis = 0)
    number_of_less_than_or_equal_to_constraints += tf.count_nonzero(input_tensor = condition)
    
    modified_constraint_inequality_direction_vector = tf.where(condition = condition, x = tf.ones(shape = [number_of_constraints], dtype = tf.int64), y = constraint_inequality_direction_vector[0:number_of_constraints])
    
    condition = tf.reduce_all(input_tensor = [greater_than_or_equal_to_mask, tf.logical_not(x = negative_constraint_constant_mask)], axis = 0)
    number_of_greater_than_or_equal_to_constraints += tf.count_nonzero(input_tensor = condition)
    
    # if originally equal to
    condition = equal_to_mask
    number_of_equal_to_constraints = tf.count_nonzero(input_tensor = condition)
    
    # if originally less than or equal to
    condition = tf.reduce_all(input_tensor = [less_than_or_equal_to_mask, negative_constraint_constant_mask], axis = 0)
    number_of_greater_than_or_equal_to_constraints += tf.count_nonzero(input_tensor = condition)
    
    modified_constraint_inequality_direction_vector = tf.where(condition = condition, x = -tf.ones(shape = [number_of_constraints], dtype = tf.int64), y = constraint_inequality_direction_vector[0:number_of_constraints])
    
    condition = tf.reduce_all(input_tensor = [less_than_or_equal_to_mask, tf.logical_not(x = negative_constraint_constant_mask)], axis = 0)
    number_of_less_than_or_equal_to_constraints += tf.count_nonzero(input_tensor = condition)
    
    modified_constraint_inequality_direction_vector = tf.concat(values = [modified_constraint_inequality_direction_vector, tf.ones(shape = [number_of_variables_required_to_be_binary], dtype = tf.int64)], axis = 0)
    
    return number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, modified_constraint_inequality_direction_vector

In [19]:
# This function initializes the current tableau row and column counts
def initialize_current_tableau_row_and_column_counts(number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables):
    tableau_current_size_numerator = tf.cast(x = tf.where(condition = tf.greater(x = number_of_artificial_variables, y = 0), x = number_of_constraints + number_of_variables_required_to_be_binary + 2, y = number_of_constraints + number_of_variables_required_to_be_binary + 1), dtype = tf.int64)
    tableau_current_size_denominator = number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables + 1

    tableau_current_size = tf.stack(values = [tableau_current_size_numerator, tableau_current_size_denominator], axis = 0)

    return tableau_current_size

In [20]:
# This function creates the simplex tableau matrix
def create_tableau_matrix(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix):
    """
        Tableau generic format is:

        con\var   |   b    x1    x2    ...   xn    s1       s2       ...   sq       A1    A2    ...      Ar
        ------------------------------------------------------------------------------------------------------------------------
        con1      |   b1   x11   x11   ...   x1n   0/1/-1   0/1/-1   ...   0/1/-1   0/1   0/1   ...      0/1
        con2      |   b2   x21   x22   ...   x2n   0/1/-1   0/1/-1   ...   0/1/-1   0/1   0/1   ...      0/1
        .         |   .    .     .     ...   .     .        .        ...   .        .     .     ...      .
        .         |   .    .     .     ...   .     .        .        ...   .        .     .     ...      .
        .         |   .    .     .     ...   .     .        .        ...   .        .     .     ...      .
        conm      |   bm   xm1   xm2   ...   xmn   0/1/-1   0/1/-1   ...   0/1/-1   0/1   0/1   ...      0/1
        z         |   z0   -c1   -c2   ...   0     0        0        ...   0        0     0     ...      0
        zA        |   0    0     0     ...   0     0        0        ...   0        -1    -1    ...      -1
    """

    # First add b constant vector
    b_constant_vector = tf.stack(values = [tf.expand_dims(input = tf.where(condition = tf.less(x = constraint_constant_vector[0, :], y = 0), x = -constraint_constant_vector[0, :], y = constraint_constant_vector[0, :]), axis = -1), tf.expand_dims(input = constraint_constant_vector[1, :], axis = -1)], axis = 0)
    
    # Next add b constant vector for binary required variables
    b_constant_vector_binary = tf.ones(shape = [2, number_of_variables_required_to_be_binary, 1], dtype = tf.int64)

    # Next add A matrix
    a_matrix = tf.stack(values = [tf.where(condition = tf.less(x = constraint_constant_vector[0, :], y = 0), x = -constraint_coefficient_matrix[0, :, :], y = constraint_coefficient_matrix[0, :, :]), constraint_coefficient_matrix[1, :, :]], axis = 0)

    # Next add A matrix for binary required variables
    binary_mask = tf.equal(x = variable_special_requirements[:], y = 2)
    a_matrix_binary_numerator = tf.boolean_mask(tensor = tf.eye(num_rows = number_of_variables, dtype = tf.int64), mask = binary_mask, axis = 0)
    a_matrix_binary_denominator = tf.boolean_mask(tensor = tf.ones(shape = [number_of_variables, number_of_variables], dtype = tf.int64), mask = binary_mask, axis = 0)
    a_matrix_binary = tf.stack(values = [a_matrix_binary_numerator, a_matrix_binary_denominator], axis = 0)
    a_matrix_binary = tf.reshape(tensor = a_matrix_binary, shape = [2, number_of_variables_required_to_be_binary, number_of_variables])
            
    # Next add slack/surplus variable matrix
    condition = tf.equal(x = constraint_inequality_direction_vector[:], y = 1)
    true_array = tf.eye(num_rows = number_of_constraints + number_of_variables_required_to_be_binary, dtype = tf.int64)
    false_array = -tf.eye(num_rows = number_of_constraints + number_of_variables_required_to_be_binary, dtype = tf.int64)
    slack_surplus_matrix = tf.stack(values = [tf.where(condition = condition, x = true_array, y = false_array), tf.ones(shape = [number_of_constraints + number_of_variables_required_to_be_binary, number_of_constraints + number_of_variables_required_to_be_binary], dtype = tf.int64)], axis = 0)
            
    # Next add artificial variable matrix
    condition = tf.not_equal(x = constraint_inequality_direction_vector[:], y = 1)
    true_array = tf.eye(num_rows = number_of_constraints + number_of_variables_required_to_be_binary, dtype = tf.int64)
    false_array = tf.zeros(shape = [number_of_constraints + number_of_variables_required_to_be_binary, number_of_constraints + number_of_variables_required_to_be_binary], dtype = tf.int64)
    temp_artificial_variable_matrix = tf.stack(values = [tf.where(condition = condition, x = true_array, y = false_array), tf.ones(shape = [number_of_constraints + number_of_variables_required_to_be_binary, number_of_constraints + number_of_variables_required_to_be_binary], dtype = tf.int64)], axis = 0)
    artificial_variable_matrix = tf.boolean_mask(tensor = temp_artificial_variable_matrix, mask = condition, axis = 2)
    
    # Next add original -cT vector
    original_ct_vector = tf.expand_dims(input = tf.stack(values = [tf.where(condition = tf.equal(x = maximization_problem, y = 1), x = -objective_function_coefficient_vector[0, :], y = objective_function_coefficient_vector[0, :]), tf.ones(shape = [number_of_variables], dtype = tf.int64)], axis = 0), axis = 1)
    
    # Next add artificial -cT vector
    artificial_ct_vector = tf.expand_dims(input = tf.stack(values = [-tf.ones(shape = [number_of_artificial_variables], dtype = tf.int64), tf.ones(shape = [number_of_artificial_variables], dtype = tf.int64)], axis = 0), axis = 1)

    # Lastly add initial z
    initial_z = tf.reshape(tensor = objective_function_initial_constant, shape = [2, 1, 1])
    
    # Now construct tableau with all of the pieces
    b_constant_column = tf.concat(values = [b_constant_vector, b_constant_vector_binary], axis = 1)
    
    a_matrix_full = tf.concat(values = [a_matrix, a_matrix_binary], axis = 1)
    
    non_artificial_tableau_above_objective_rows = tf.concat(values = [b_constant_column, a_matrix_full, slack_surplus_matrix], axis = 2)
    
    artificial_tableau_above_objective_rows = artificial_variable_matrix
    
    objective_row = tf.concat(values = [initial_z, original_ct_vector, tf.stack(values = [tf.zeros(shape = [1, number_of_slack_surplus_variables], dtype = tf.int64), tf.ones(shape = [1, number_of_slack_surplus_variables], dtype = tf.int64)], axis = 0)], axis = 2)
    
    non_artificial_tableau = tf.concat(values = [non_artificial_tableau_above_objective_rows, objective_row], axis = 1)
    
    artificial_tableau = tf.concat(values = [artificial_tableau_above_objective_rows, tf.stack(values = [tf.zeros(shape = [1, number_of_artificial_variables], dtype = tf.int64), tf.ones(shape = [1, number_of_artificial_variables], dtype = tf.int64)], axis = 0)], axis = 1)
    
    combined_tableau = tf.concat(values = [non_artificial_tableau, artificial_tableau], axis = 2)
    
    def finish_tableau_matrix_with_artificial_variables(combined_tableau, number_of_variables, number_of_slack_surplus_variables, artificial_ct_vector):
        artificial_objective_row = tf.concat(values = [tf.stack(values = [tf.zeros(shape = [1, 1 + number_of_variables + number_of_slack_surplus_variables], dtype = tf.int64), tf.ones(shape = [1, 1 + number_of_variables + number_of_slack_surplus_variables], dtype = tf.int64)], axis = 0), artificial_ct_vector], axis = 2)
        
        return tf.concat(values = [combined_tableau, artificial_objective_row], axis = 1)
    
    def finish_tableau_matrix(combined_tableau):
        return combined_tableau
    
    tableau_matrix = tf.cond(pred = tf.greater(x = number_of_artificial_variables, y = 0), true_fn = lambda: finish_tableau_matrix_with_artificial_variables(combined_tableau, number_of_variables, number_of_slack_surplus_variables, artificial_ct_vector), false_fn = lambda: finish_tableau_matrix(combined_tableau))
    
    number_of_constraints += number_of_variables_required_to_be_binary
    
    return number_of_constraints, tableau_matrix

In [21]:
# This function creates the basic variables which tell which basis we currently are in
def create_basic_variables(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix):
    condition = tf.greater(x = tableau_matrix[0, :number_of_constraints, 1:1 + number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables], y =0)
    
    number_of_positive_elements = tf.count_nonzero(input_tensor = condition, axis = 0)
    
    positive_element_column_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables, dtype = tf.int64), mask = tf.equal(x = number_of_positive_elements, y = 1))
    
    positive_element_column_values = tf.gather(params = condition, indices = positive_element_column_indices, axis = 1)
    
    positive_element_row_indices = tf.squeeze(input = tf.stack(values = tf.map_fn(fn = lambda i: tf.where(positive_element_column_values[:, i]), elems = tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64)), axis = 0))
    
    basic_variables = tf.scatter_nd(indices = tf.expand_dims(input = positive_element_row_indices, axis = -1), updates = positive_element_column_indices + 1, shape = [number_of_constraints])

    return basic_variables

In [22]:
# This function creates the basic feasible solution to keep track of the best variable values in the current basis
def create_basic_feasible_solution(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, tableau_matrix):
    basic_feasible_solution_basic_variable_values = tf.stack(values = tf.map_fn(fn = lambda x: rational_division(tableau_matrix[0, x[0], 0], tableau_matrix[1, x[0], 0], tableau_matrix[0, x[0], x[1]], tableau_matrix[1, x[0], x[1]]), elems = (tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), basic_variables[:number_of_constraints])), axis = 0)

    basic_feasible_solution_numerator = tf.scatter_nd(indices = tf.expand_dims(input = basic_variables, axis = -1) - 1, updates = basic_feasible_solution_basic_variable_values[0], shape = [number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables])
        
    basic_feasible_solution_denominator = tf.scatter_nd(indices = tf.expand_dims(input = basic_variables, axis = -1) - 1, updates = basic_feasible_solution_basic_variable_values[1], shape = [number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables])
    
    basic_feasible_solution_denominator_fixed = tf.where(condition = tf.equal(x = basic_feasible_solution_denominator, y = 0), x = tf.ones(shape = [number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables], dtype = tf.int64), y = basic_feasible_solution_denominator)
    
    basic_feasible_solution = tf.stack(values = [basic_feasible_solution_numerator, basic_feasible_solution_denominator_fixed], axis = 0)

    return basic_feasible_solution

In [23]:
# This function writes to disk the initial LP array values
def write_initial_LP_values(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix):
    print("write_initial_LP_values: objective function optimal value = \n{}".format(tf.cast(x = tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64)))
    print("write_initial_LP_values: basic variables = \n{}".format(basic_variables))
    print("write_initial_LP_values: basic feasible solution = \n{}".format(tf.cast(x = basic_feasible_solution[0, :], dtype = tf.float64) / tf.cast(x = basic_feasible_solution[1, :], dtype = tf.float64)))
    print("write_initial_LP_values: tableau matrix = \n{}".format(tf.cast(x = tableau_matrix[0, :, :], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, :, :], dtype = tf.float64)))
    
    return

In [24]:
# This function prints the initial constraint, variable, etc. counts
def print_initial_counts(number_of_constraints, number_of_variables, number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size):
    print("print_initial_counts: number_of_constraints = {} & number_of_variables = {}".format(number_of_constraints, number_of_variables))
    print("print_initial_counts: number_of_less_than_or_equal_to_constraints = {}, number_of_equal_to_constraints = {}, number_of_greater_than_or_equal_to_constraints = {}".format(number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints))
    print("print_initial_counts: number_of_slack_surplus_variables = {} & number_of_artificial_variables = {}".format(number_of_slack_surplus_variables, number_of_artificial_variables))
    print("print_initial_counts: tableau_current_size = {}".format(tableau_current_size))
    
    return

In [25]:
# This function writes to disk the final LP array values
def write_final_LP_values(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix):
    print("write_final_LP_values: objective function optimal value = \n{}".format(tf.cast(x = tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64)))
    print("write_final_LP_values: basic variables = \n{}".format(basic_variables))
    print("write_final_LP_values: basic feasible solution = \n{}".format(tf.cast(x = basic_feasible_solution[0, :], dtype = tf.float64) / tf.cast(x = basic_feasible_solution[1, :], dtype = tf.float64)))
    print("write_final_LP_values: tableau matrix = \n{}".format(tf.cast(x = tableau_matrix[0, :, :], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, :, :], dtype = tf.float64)))
    
    return

In [26]:
# This function writes to disk the final MILP array values
def write_final_MILP_values(optimal_objective_function_value, optimal_variable_values):
    print("write_final_MILP_values: optimal_objective_function_value = \n{}".format(tf.cast(x = optimal_objective_function_value[0], dtype = tf.float64) / tf.cast(x = optimal_objective_function_value[1], dtype = tf.float64)))
    print("write_final_MILP_values: optimal_variable_values = \n{}".format(tf.cast(x = optimal_variable_values[0, :], dtype = tf.float64) / tf.cast(x = optimal_variable_values[1, :], dtype = tf.float64)))
    
    return

In [27]:
# This function updates the optimal objective function and variable solution
def update_optimal_objective_function_and_variable_solution(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix):
    optimal_objective_function_value = tf.where(condition = tf.equal(maximization_problem, y = 1), x = tableau_matrix[:, number_of_constraints, 0], y = tf.stack(values = [-tableau_matrix[0, number_of_constraints, 0], tableau_matrix[1, number_of_constraints, 0]], axis = 0))

    optimal_variable_values = basic_feasible_solution[:, :number_of_variables]

    return optimal_objective_function_value, optimal_variable_values

# Simplex

In [28]:
# This function finds the optimal solution for the given variables and constraints
def simplex_algorithm(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
    error_code = tf.constant(value = 0, dtype = tf.int64)

    #*********************************************************************************
    #*********************************** PHASE 1 *************************************
    #*********************************************************************************

    def have_artificial_variables(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
        # This function transforms the tableau by removing artificial variables to obtain a basic feasible solution
        error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = simplex_phase_1(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size)
        tableau_current_size = tf.stack(values = [tableau_current_size[0] - 1, tableau_current_size[1]], axis = 0) # decrement current rows since we've eliminated artificial objective function row
        tableau_matrix = tableau_matrix[:, 0:tableau_current_size[0], 0:tableau_current_size[1]]
        
        return error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size
    
    def dont_have_artificial_variables(error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size):
        return error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size

    error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = \
        tf.cond(
            pred = tf.greater(x = number_of_artificial_variables, y = 0), 
            true_fn = lambda: have_artificial_variables(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size), 
            false_fn = lambda: dont_have_artificial_variables(error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size))

    #*********************************************************************************
    #*********************************** PHASE 2 *************************************
    #*********************************************************************************

    def ready_for_simplex_phase_2(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
        # This function starts from an basic feasible solution and iterates toward the optimal solution
        error_code, basic_variables, basic_feasible_solution, tableau_matrix = simplex_phase_2(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size)
        
        return error_code, basic_variables, basic_feasible_solution, tableau_matrix
    
    def not_ready_for_simplex_phase_2(error_code, basic_variables, basic_feasible_solution, tableau_matrix):
        return error_code, basic_variables, basic_feasible_solution, tableau_matrix
    
    error_code, basic_variables, basic_feasible_solution, tableau_matrix = tf.cond(
        pred = tf.equal(x = error_code, y = 0), 
        true_fn = lambda: ready_for_simplex_phase_2(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size), 
        false_fn = lambda: not_ready_for_simplex_phase_2(error_code, basic_variables, basic_feasible_solution, tableau_matrix))

    #*********************************************************************************
    #*********************************** SOLUTION ************************************
    #*********************************************************************************

    def basic_feasible_solution_exists(number_of_variables, number_of_slack_surplus_variables, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix):
        basic_feasible_solution = update_basic_feasible_solution(number_of_variables + number_of_slack_surplus_variables, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix)
        
        return basic_feasible_solution
    
    def no_basic_feasible_solution_exists(basic_feasible_solution):
        return basic_feasible_solution
    
    basic_feasible_solution = tf.cond(
        pred = tf.equal(x = error_code, y = 0), 
        true_fn = lambda: basic_feasible_solution_exists(number_of_variables, number_of_slack_surplus_variables, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix), 
        false_fn = lambda: no_basic_feasible_solution_exists(basic_feasible_solution))

    return error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size

In [29]:
# This function transforms the tableau by removing artificial variables to obtain a basic feasible solution
def simplex_phase_1(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
    error_code = tf.constant(value = 0, dtype = tf.int64)

    # PHASE I (first find the feasible region)

    # Remove artificial variables from objective function
    artificial_variable_col_index = tf.range(start = number_of_variables + number_of_slack_surplus_variables + 1, limit = number_of_variables + number_of_slack_surplus_variables + 1 + number_of_artificial_variables, dtype = tf.int64)

    artificial_variable_row_numerator_mask = tf.map_fn(fn = lambda j: tf.equal(x = tableau_matrix[0, 0:tableau_current_size[0] - 2, j], y = 1), elems = artificial_variable_col_index, dtype = tf.bool)
    artificial_variable_row_denominator_mask = tf.map_fn(fn = lambda j: tf.equal(x = tableau_matrix[1, 0:tableau_current_size[0] - 2, j], y = 1), elems = artificial_variable_col_index, dtype = tf.bool)    
    
    condition = tf.reduce_all(input_tensor = [artificial_variable_row_numerator_mask, artificial_variable_row_denominator_mask], axis = 0)
    
    tiled_constraint_indices = tf.reshape(tensor = tf.tile(input = tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), multiples = [number_of_artificial_variables]), shape = [number_of_artificial_variables, number_of_constraints])
    
    artificial_variable_row_index = tf.boolean_mask(tensor = tiled_constraint_indices, mask = condition, axis = 0)
    
    updated_artificial_objective_row = tableau_matrix[:, number_of_constraints + 1, :]
    
    def updated_artificial_objective_row_while_loop_condition(i, number_of_artificial_variables, tableau_matrix, artificial_variable_row_index, tableau_current_size, updated_artificial_objective_row):
        return tf.less(x = i, y = number_of_artificial_variables)
    
    def updated_artificial_objective_row_while_loop_body(i, number_of_artificial_variables, tableau_matrix, artificial_variable_row_index, tableau_current_size, updated_artificial_objective_row):
        updated_artificial_objective_row = tf.stack(values = tf.map_fn(fn = lambda j: rational_addition(updated_artificial_objective_row[0, j], updated_artificial_objective_row[1, j], tableau_matrix[0, artificial_variable_row_index[i], j], tableau_matrix[1, artificial_variable_row_index[i], j]), elems = tf.range(start = 0, limit = tableau_current_size[1], dtype = tf.int64), dtype = (tf.int64, tf.int64)), axis = 0)
        i += 1
        
        return i, number_of_artificial_variables, tableau_matrix, artificial_variable_row_index, tableau_current_size, updated_artificial_objective_row
        
    # Call tf.while_loop
    i0 = tf.constant(value = 0, dtype = tf.int64)    
    _, _, _, _, _, updated_artificial_objective_row = tf.while_loop(cond = updated_artificial_objective_row_while_loop_condition, body = updated_artificial_objective_row_while_loop_body, loop_vars = [i0, number_of_artificial_variables, tableau_matrix, artificial_variable_row_index, tableau_current_size, updated_artificial_objective_row], shape_invariants = [i0.get_shape(), number_of_artificial_variables.get_shape(), tableau_matrix.get_shape(), artificial_variable_row_index.get_shape(), tableau_current_size.get_shape(), tf.TensorShape(dims = [2, None])])
    
    tableau_matrix = tf.concat(values = [tableau_matrix[:, :number_of_constraints + 1, :], tf.expand_dims(input = updated_artificial_objective_row, axis = 1)], axis = 1)
        
    # Update Basic Feasible Solution
    basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, tableau_current_size[0] - 2, basic_variables, basic_feasible_solution, tableau_matrix)

    # Count initial number of positive elements in the objective function row
    number_of_positive_objective_function_elements = tf.count_nonzero(input_tensor = tf.greater(x = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]], y = 0))

    def while_loop_condition(number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix):
        return tf.reduce_all(input_tensor = [tf.greater(x = number_of_positive_objective_function_elements, y = 0), tf.greater(x = number_of_artificial_variables, y = 0), tf.equal(x = error_code, y = 0)])
    
    def while_loop_body(number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix):
        # Find most positive objective function element amongst the variables which will become our entering variable
        objective_function_row = tf.cast(x = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, tableau_current_size[0] - 1, 1:tableau_current_size[1]], dtype = tf.float64)
        most_positive_objective_function_element_value = tf.reduce_max(input_tensor = objective_function_row)
        most_positive_objective_function_element_index = tf.argmax(input = objective_function_row) + 1

        def pivot_column_found(most_positive_objective_function_element_value, most_positive_objective_function_element_index, number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix):
            pivot_col_index = most_positive_objective_function_element_index

            # Search for smallest non negative ratio of bi / aij which will be departing variable
            condition = tf.greater(x = tableau_matrix[0, 0:tableau_current_size[0] - 2, pivot_col_index], y = 0) # pivot element needs to be positive
            b_a_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = tableau_current_size[0] - 2), mask = condition)
            b_a_ratio = tf.stack(values = tf.map_fn(fn = lambda i: rational_division(tableau_matrix[0, i, 0], tableau_matrix[1, i, 0], tableau_matrix[0, i, pivot_col_index], tableau_matrix[1, i, pivot_col_index]), elems = b_a_indices, dtype = (tf.int64, tf.int64)), axis = 0)
            b_a_ratio_double = tf.cast(x = b_a_ratio[0, :], dtype = tf.float64) / tf.cast(x = b_a_ratio[1, :], dtype = tf.float64)

            # Non-negative
            condition = tf.greater_equal(x = b_a_ratio_double, y = 0)
            non_negative_b_a_ratio_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = tf.shape(input = b_a_ratio_double, out_type = tf.int64)[0], dtype = tf.int64), mask = condition)
            non_negative_b_a_ratio_values_double = tf.boolean_mask(tensor = b_a_ratio_double, mask = condition)
            non_negative_b_a_ratio_values = tf.boolean_mask(tensor = b_a_ratio, mask = condition, axis = 1)

            smallest_non_negative_ratio_value = tf.reduce_min(input_tensor = non_negative_b_a_ratio_values_double)
            smallest_non_negative_ratio_value_index = tf.argmin(input = non_negative_b_a_ratio_values_double)
            smallest_non_negative_ratio_value = non_negative_b_a_ratio_values[:, smallest_non_negative_ratio_value_index]
            smallest_non_negative_ratio_index = b_a_indices[non_negative_b_a_ratio_indices[smallest_non_negative_ratio_value_index]] # this should already be using Bland's Rule since it is taking the lowest index in ties to avoid cycles
            
            # Artificial
            condition = tf.greater_equal(x = tf.gather(params = basic_variables, indices = b_a_indices), y = number_of_variables + number_of_slack_surplus_variables)
            non_negative_b_a_ratio_artificial_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = tf.shape(input = b_a_ratio_double, out_type = tf.int64)[0], dtype = tf.int64), mask = condition)
            non_negative_b_a_ratio_artificial_values_double = tf.boolean_mask(tensor = b_a_ratio_double, mask = condition)
            non_negative_b_a_ratio_artificial_values = tf.boolean_mask(tensor = b_a_ratio, mask = condition, axis = 1)
            
            smallest_artificial_variable_non_negative_ratio_value = tf.reduce_min(input_tensor = non_negative_b_a_ratio_artificial_values_double)
            smallest_artificial_variable_non_negative_ratio_value_index = tf.argmin(input = non_negative_b_a_ratio_artificial_values_double)
            smallest_artificial_variable_non_negative_ratio_value = non_negative_b_a_ratio_artificial_values[:, smallest_artificial_variable_non_negative_ratio_value_index]
            smallest_artificial_variable_non_negative_ratio_index = b_a_indices[non_negative_b_a_ratio_artificial_indices[smallest_artificial_variable_non_negative_ratio_value_index]] # this should already be using Bland's Rule since it is taking the lowest index in ties to avoid cycles

            condition_pivot_row_found = tf.not_equal(x = tf.size(input = smallest_non_negative_ratio_value, out_type = tf.int64), y = 0)

            def pivot_row_found(pivot_col_index, smallest_non_negative_ratio_value, smallest_non_negative_ratio_index, smallest_artificial_variable_non_negative_ratio_value, smallest_artificial_variable_non_negative_ratio_index, error_code, number_of_constraints, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
                condition_no_more_artificial_variables_remaining = tf.equal(x = smallest_artificial_variable_non_negative_ratio_index, y = 0)
                condition_different_non_negative_ratio_indices = tf.not_equal(x = smallest_non_negative_ratio_index, y = smallest_artificial_variable_non_negative_ratio_index)
                condition_same_non_negative_ratio_values = tf.reduce_all(input_tensor = [tf.equal(x = smallest_non_negative_ratio_value[0], y = smallest_artificial_variable_non_negative_ratio_value[0]), tf.equal(x = smallest_non_negative_ratio_value[1], y = smallest_artificial_variable_non_negative_ratio_value[1])])
                
                # IF LOWEST RATIO OCCURS BOTH IN AN ARTIFICIAL VARIABLE ROW AND A NON-NEGATIVE VARIABLE ROW THEN MUST CHOOSE PIVOT ROW AS NEGATIVE VARIABLE ROW
                pivot_row_index = tf.where(condition = tf.reduce_all(input_tensor = [tf.logical_not(x = condition_no_more_artificial_variables_remaining), condition_different_non_negative_ratio_indices, condition_same_non_negative_ratio_values]), x = smallest_artificial_variable_non_negative_ratio_index, y = smallest_non_negative_ratio_index)
            
                pivot_value = tableau_matrix[:, pivot_row_index, pivot_col_index]

                condition = tf.greater_equal(x = basic_variables[pivot_row_index], y = number_of_variables + number_of_slack_surplus_variables) # if basic variable is an artificial variable
                
                artificial_column = tf.reshape(tensor = tf.constant(value = [], dtype = tf.int64), shape = [2, tableau_current_size[0], 0])
                
                def basic_variable_is_artificial(number_of_artificial_variables, artificial_column, tableau_current_size):
                    number_of_artificial_variables -= 1
                    artificial_column = tf.reshape(tensor = tableau_matrix[:, :, tableau_current_size[1] - 1], shape = [2, tableau_current_size[0], 1])
                    tableau_current_size = tf.stack(values = [tableau_current_size[0], tableau_current_size[1] - 1], axis = 0)
                    
                    return number_of_artificial_variables, artificial_column, tableau_current_size
                
                def basic_variable_is_not_artificial(number_of_artificial_variables, artificial_column, tableau_current_size):
                    return number_of_artificial_variables, artificial_column, tableau_current_size
                
                number_of_artificial_variables, artificial_column, tableau_current_size = tf.cond(pred = condition, true_fn = lambda: basic_variable_is_artificial(number_of_artificial_variables, artificial_column, tableau_current_size), false_fn = lambda: basic_variable_is_not_artificial(number_of_artificial_variables, artificial_column, tableau_current_size))

                # Remove departing variable from and add entering variable to basic variables
                pivot_row_index_condition = tf.equal(x = tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), y = pivot_row_index)
                basic_variables = tf.where(condition = pivot_row_index_condition, x = tf.tile(input = [pivot_col_index], multiples = [number_of_constraints]), y = basic_variables)

                # This function performs Gauss-Jordan Elimination on the pivot column
                tableau_matrix = pivot_column_gauss_jordan_elimnation(tableau_current_size, pivot_row_index, pivot_col_index, pivot_value, tableau_matrix)
                
                # If we dropped an artificial column, add artificial column back on to tableau for basic feasible solution due to basic variable index
                tableau_matrix = tf.concat(values = [tableau_matrix, artificial_column], axis = 2)

                # This function updates the basic feasible solution
                basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, tableau_current_size[0] - 2, basic_variables, basic_feasible_solution, tableau_matrix)

                # Count again the number of variables that are positive still
                number_of_positive_objective_function_elements = tf.count_nonzero(input_tensor = tf.greater(x = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]], y = 0))

                return number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix

            def pivot_row_not_found(pivot_col_index, smallest_non_negative_ratio_value, smallest_non_negative_ratio_index, smallest_artificial_variable_non_negative_ratio_value, smallest_artificial_variable_non_negative_ratio_index, error_code, number_of_constraints, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
                error_code = tf.constant(value = 1, dtype = tf.int64) # infeasible
            
                return number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix
            
            # If pivot row found or not
            number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix = \
                tf.cond(
                    pred = condition_pivot_row_found, 
                    true_fn = lambda: pivot_row_found(pivot_col_index, smallest_non_negative_ratio_value, smallest_non_negative_ratio_index, smallest_artificial_variable_non_negative_ratio_value, smallest_artificial_variable_non_negative_ratio_index, error_code, number_of_constraints, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size), 
                    false_fn = lambda: pivot_row_not_found(pivot_col_index, smallest_non_negative_ratio_value, smallest_non_negative_ratio_index, smallest_artificial_variable_non_negative_ratio_value, smallest_artificial_variable_non_negative_ratio_index, error_code, number_of_constraints, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size))
            
            return number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix
        
        def pivot_column_not_found(number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix):
            error_code = tf.constant(value = 1, dtype = tf.int64) # infeasible
            number_of_positive_objective_function_elements = tf.constant(value = 0, dtype = tf.int64)
            
            return number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix

        # If pivot column was found or not
        number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix = \
            tf.cond(
                pred = tf.greater(x = most_positive_objective_function_element_value, y = 0), 
                true_fn = lambda: pivot_column_found(most_positive_objective_function_element_value, most_positive_objective_function_element_index, number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix), 
                false_fn = lambda: pivot_column_not_found(number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix))
            
        return number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix
    
    # Call tf.while_loop
    number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix = \
        tf.while_loop(
            cond = while_loop_condition, 
            body = while_loop_body, 
            loop_vars = [number_of_positive_objective_function_elements, number_of_artificial_variables, error_code, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix], 
            shape_invariants = [number_of_positive_objective_function_elements.get_shape(), number_of_artificial_variables.get_shape(), error_code.get_shape(), tableau_current_size.get_shape(), basic_variables.get_shape(), tf.TensorShape(dims = [2, None]), tf.TensorShape(dims = [2, None, None])])

    def artificial_variables_remaining_at_end_of_phase_1(number_of_artificial_variables, error_code):
        error_code = tf.constant(value = 1, dtype = tf.int64) # infeasible
        
        return error_code
        
    def no_artificial_variables_remaining_at_end_of_phase_1(error_code):
        return error_code
        
    error_code = tf.cond(
        pred = tf.greater(x = number_of_artificial_variables, y = 0), 
        true_fn = lambda: artificial_variables_remaining_at_end_of_phase_1(number_of_artificial_variables, error_code), 
        false_fn = lambda: no_artificial_variables_remaining_at_end_of_phase_1(error_code))

    return error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size

In [30]:
# This function starts from an basic feasible solution and iterates toward the optimal solution
def simplex_phase_2(number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
    error_code = tf.constant(value = 0, dtype = tf.int64)
    old_optimum = tf.constant(value = np.finfo(np.float64).min, dtype = tf.float64)

    # PHASE II (find the optimal solution in the feasible region we found above)

    # Count initial number of negative elements in the objective function row
    number_of_negative_objective_function_elements = tf.count_nonzero(input_tensor = tf.less(x = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]], y = 0))
    
    def while_loop_condition(number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix):
        # while there is still a optimal solution
        return tf.logical_and(x = tf.greater(x = number_of_negative_objective_function_elements, y = 0), y = tf.equal(x = error_code, y = 0))
    
    def while_loop_body(number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix):
        # Search for most negative element in objective function row
        objective_function_row = tf.cast(x = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]], dtype = tf.float64) / tf.cast(tableau_matrix[1, tableau_current_size[0] - 1, 1:tableau_current_size[1]], dtype = tf.float64)
        most_negative_objective_function_element_value = tf.reduce_min(input_tensor = objective_function_row)
        most_negative_objective_function_element_index = tf.argmin(input = objective_function_row) + 1

        condition_pivot_column_found = tf.less(x = most_negative_objective_function_element_value, y = 0)
        
        def pivot_column_found(old_optimum, most_negative_objective_function_element_index, error_code, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
            pivot_col_index = most_negative_objective_function_element_index

            # Check to see if the problem is unbounded
            number_of_positive_elements = tf.count_nonzero(input_tensor = tf.greater(x = tableau_matrix[0, 0:tableau_current_size[0] - 1, pivot_col_index], y = 0))

            condition_still_positive_elements = tf.greater(x = number_of_positive_elements, y = 0) # if the problem is bounded still
            
            def still_positive_elements(old_optimum, pivot_col_index, error_code, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
                # Search for smallest non negative ratio of bi / aij
                condition = tf.greater(x = tableau_matrix[0, 0:tableau_current_size[0] - 1, pivot_col_index], y = 0) # pivot element needs to be positive
                b_a_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = tableau_current_size[0] - 1, dtype = tf.int64), mask = condition)
                b_a_numerator = tf.boolean_mask(tensor = tf.cast(x = tableau_matrix[0, 0:tableau_current_size[0] - 1, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, 0:tableau_current_size[0] - 1, 0], dtype = tf.float64), mask = condition)
                b_a_denominator = tf.boolean_mask(tensor = tf.cast(x = tableau_matrix[0, 0:tableau_current_size[0] - 1, pivot_col_index], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, 0:tableau_current_size[0] - 1, pivot_col_index], dtype = tf.float64), mask = condition)
                b_a_ratio = b_a_numerator / b_a_denominator

                non_negative_b_a_ratio_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = tf.shape(input = b_a_ratio, out_type = tf.int64)[0], dtype = tf.int64), mask = tf.greater_equal(x = b_a_ratio, y = 0))
                non_negative_b_a_ratio_values = tf.boolean_mask(tensor = b_a_ratio, mask = tf.greater_equal(x = b_a_ratio, y = 0))

                smallest_non_negative_ratio_value = tf.reduce_min(input_tensor = non_negative_b_a_ratio_values)
                smallest_non_negative_ratio_index = b_a_indices[non_negative_b_a_ratio_indices[tf.argmin(input = non_negative_b_a_ratio_values)]] # this should already be using Bland's Rule since it is taking the lowest index in ties to avoid cycles
                
                condition_pivot_row_found = tf.not_equal(x = tf.size(input = smallest_non_negative_ratio_value, out_type = tf.int64), y = 0)
                
                def pivot_row_found(old_optimum, pivot_col_index, smallest_non_negative_ratio_index, error_code, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size):
                    pivot_row_index = smallest_non_negative_ratio_index
                    
                    pivot_value = tableau_matrix[:, pivot_row_index, pivot_col_index]

                    # Remove departing variable from and add entering variable to basic variables
                    basic_variables = tf.where(condition = tf.equal(x = tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), y = pivot_row_index), x = tf.ones(shape = [number_of_constraints], dtype = tf.int64) * pivot_col_index, y = basic_variables)

                    # This function performs Gauss-Jordan Elimination on the pivot column
                    tableau_matrix = pivot_column_gauss_jordan_elimnation(tableau_current_size, pivot_row_index, pivot_col_index, pivot_value, tableau_matrix)

                    basic_variables = tf.where(condition = tf.equal(x = tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), y = pivot_row_index), x = tf.ones(shape = [number_of_constraints], dtype = tf.int64) * pivot_col_index, y = basic_variables)
                    
                    # This function updates the basic feasible solution
                    basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, tableau_current_size[0] - 1, basic_variables, basic_feasible_solution, tableau_matrix)
                                               
                    # Count new number of negative elements in the objective function row
                    number_of_negative_objective_function_elements = tf.count_nonzero(input_tensor = tf.less(x = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]], y = 0))
                                               
                    def old_optimum_true_fn(old_optimum):
                        return old_optimum
                    
                    def old_optimum_false_fn(current_optimum):
                        return current_optimum
                    
                    current_optimum = tf.cast(x = tableau_matrix[0, tableau_current_size[0] - 1, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, tableau_current_size[0] - 1, 0], dtype = tf.float64)
                    
                    condition = tf.equal(x = old_optimum, y = current_optimum)
                    
                    old_optimum = tf.cond(pred = condition, true_fn = lambda: old_optimum_true_fn(old_optimum), false_fn = lambda: old_optimum_false_fn(current_optimum))
                    
                    return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
                
                def pivot_row_not_found(old_optimum, basic_variables, basic_feasible_solution, tableau_matrix):
                    error_code = tf.constant(value = 1, dtype = tf.int64) # infeasible
                    number_of_negative_objective_function_elements = tf.constant(value = 0, dtype = tf.int64)
                    
                    return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
                
                number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix = \
                    tf.cond(
                        pred = condition_pivot_row_found, 
                        true_fn = lambda: pivot_row_found(old_optimum, pivot_col_index, smallest_non_negative_ratio_index, error_code, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size), 
                        false_fn = lambda: pivot_row_not_found(old_optimum, basic_variables, basic_feasible_solution, tableau_matrix))
                
                return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
            
            def not_still_positive_elements(old_optimum, basic_variables, basic_feasible_solution, tableau_matrix):
                error_code = tf.constant(value = 2, dtype = tf.int64) # unbounded
                number_of_negative_objective_function_elements = tf.constant(value = 0, dtype = tf.int64)
                
                return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
            
            number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix = \
                tf.cond(
                    pred = condition_still_positive_elements, 
                    true_fn = lambda: still_positive_elements(old_optimum, pivot_col_index, error_code, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size), 
                    false_fn = lambda: not_still_positive_elements(old_optimum, basic_variables, basic_feasible_solution, tableau_matrix))
            
            return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
            
        def pivot_column_not_found(old_optimum, basic_variables, basic_feasible_solution, tableau_matrix):
            error_code = tf.constant(value = 1, dtype = tf.int64) # infeasible
            number_of_negative_objective_function_elements = tf.constant(value = 0, dtype = tf.int64)
            
            return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
            
        number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix = \
            tf.cond(
                pred = condition_pivot_column_found, 
                true_fn = lambda: pivot_column_found(old_optimum, most_negative_objective_function_element_index, error_code, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size), 
                false_fn = lambda: pivot_column_not_found(old_optimum, basic_variables, basic_feasible_solution, tableau_matrix))
            
        return number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix
    
    # Call tf.while_loop
    number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix = \
        tf.while_loop(
            cond = while_loop_condition, 
            body = while_loop_body, 
            loop_vars = [number_of_negative_objective_function_elements, error_code, old_optimum, basic_variables, basic_feasible_solution, tableau_matrix])

    return error_code, basic_variables, basic_feasible_solution, tableau_matrix

In [31]:
# This function performs Gauss-Jordan Elimination on the pivot column
def pivot_column_gauss_jordan_elimnation(matrix_size, pivot_row_index, pivot_col_index, pivot_value, tableau_matrix):
    # Perform Gauss-Jordan Elimination on pivot column around pivot row
    number_of_rows = matrix_size[0]
    number_of_columns = matrix_size[1]
    
    # First take care of pivot row
    condition = tf.reduce_any(input_tensor = tf.not_equal(x = tableau_matrix[:, pivot_row_index, pivot_col_index], y = 1))
    
    true_array = tf.stack(values = tf.map_fn(fn = lambda i: rational_division(tableau_matrix[0, pivot_row_index, i], tableau_matrix[1, pivot_row_index, i], pivot_value[0], pivot_value[1]), elems = tf.range(start = 0, limit = number_of_columns, dtype = tf.int64), dtype = (tf.int64, tf.int64)), axis = 0)
    
    false_array = tableau_matrix[:, pivot_row_index, :number_of_columns]
    
    tableau_matrix_pivot_row_fixed = tf.where(condition = condition, x = true_array, y = false_array)
    
    # Update tableau matrix with updated pivot row
    row_indices = tf.range(start = 0, limit = number_of_rows, dtype = tf.int64)

    not_pivot_row_condition = tf.not_equal(x = row_indices,  y = pivot_row_index)

    row_indices_minus_pivot = tf.boolean_mask(tensor = row_indices, mask = not_pivot_row_condition)
    
    tableau_without_pivot_row = tf.boolean_mask(tensor = tableau_matrix[:, :number_of_rows, :number_of_columns], mask = not_pivot_row_condition, axis = 1)
    
    indices_pivot_row = tf.cast(x = tf.stack(values = [[pivot_row_index], [pivot_row_index + number_of_rows]], axis = 0), dtype = tf.int32)
    
    indices_not_pivot_row = tf.cast(x = tf.stack(values = [row_indices_minus_pivot, row_indices_minus_pivot + number_of_rows], axis = 0), dtype = tf.int32)
    
    indices = [indices_pivot_row, indices_not_pivot_row]
    
    data_pivot_row = tf.expand_dims(input = tableau_matrix_pivot_row_fixed, axis = 1)
    
    data_not_pivot_row = tableau_without_pivot_row
    
    data = [data_pivot_row, data_not_pivot_row]
    
    dynamic_stitch = tf.dynamic_stitch(indices = indices, data = data)
    
    tableau_matrix = tf.reshape(tensor = dynamic_stitch, shape = [2, number_of_rows, number_of_columns])

    # Now take care of other rows
    condition = tf.reduce_all(input_tensor = [tf.not_equal(x = tf.range(start = 0, limit = number_of_rows, dtype = tf.int64), y = pivot_row_index), tf.not_equal(x = tableau_matrix[0, :number_of_rows, pivot_col_index], y =  0)], axis = 0)
    
    true_array = tableau_matrix[:, :number_of_rows, pivot_col_index]
    
    false_array = tf.stack(values = [tf.zeros(shape = [number_of_rows], dtype = tf.int64), tf.ones(shape = [number_of_rows], dtype = tf.int64)], axis = 0)

    pivot_value = tf.stack(values = [tf.where(condition = condition, x = true_array[0], y = false_array[0]), tf.where(condition = condition, x = true_array[1], y = false_array[1])], axis = 0)
    
    true_array = tf.stack(values = tf.unstack(value = tf.map_fn(fn = lambda i: tf.stack(values = tf.map_fn(fn = lambda j: rational_addition(tableau_matrix[0, i, j], tableau_matrix[1, i, j], -pivot_value[0, i] * tableau_matrix[0, pivot_row_index, j], pivot_value[1, i] * tableau_matrix[1, pivot_row_index, j]), elems = tf.range(start = 0, limit = number_of_columns, dtype = tf.int64), dtype = (tf.int64, tf.int64)), axis = 0), elems = tf.range(start = 0, limit = number_of_rows, dtype = tf.int64), dtype = tf.int64), axis = 1), axis = 0)
    
    false_array = tableau_matrix[:, :number_of_rows, :number_of_columns]
    
    tableau_matrix_other_rows_fixed = tf.stack(values = tf.unstack(value = tf.map_fn(fn = lambda i: tf.where(condition[i], true_array[:, i, :], false_array[:, i, :]), elems = tf.range(start = 0, limit = number_of_rows, dtype = tf.int64), dtype = tf.int64), axis = 1), axis = 0)
    
    tableau_matrix = tableau_matrix_other_rows_fixed
    
    return tableau_matrix

In [32]:
# This function updates the basic feasible solution
def update_basic_feasible_solution(number_of_total_variables, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix):
    # Update Basic Feasible Solution
    basic_feasible_solution_basic_variable_values = tf.stack(values = tf.map_fn(fn = lambda x: rational_division(tableau_matrix[0, x[0], 0], tableau_matrix[1, x[0], 0], tableau_matrix[0, x[0], x[1]], tableau_matrix[1, x[0], x[1]]), elems = (tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), basic_variables[:number_of_constraints])), axis = 0)

    basic_feasible_solution_numerator = tf.scatter_nd(indices = tf.expand_dims(input = basic_variables, axis = -1) - 1, updates = basic_feasible_solution_basic_variable_values[0], shape = [number_of_total_variables + 1])[:number_of_total_variables]
        
    basic_feasible_solution_denominator = tf.scatter_nd(indices = tf.expand_dims(input = basic_variables, axis = -1) - 1, updates = basic_feasible_solution_basic_variable_values[1], shape = [number_of_total_variables + 1])[:number_of_total_variables]
    
    basic_feasible_solution_denominator_fixed = tf.where(condition = tf.equal(x = basic_feasible_solution_denominator, y = 0), x = tf.ones(shape = [number_of_total_variables], dtype = tf.int64), y = basic_feasible_solution_denominator)
    
    basic_feasible_solution = tf.stack(values = [basic_feasible_solution_numerator, basic_feasible_solution_denominator_fixed], axis = 0)

    return basic_feasible_solution

# Branch and bound

In [33]:
# This function initiates the branch and bound
def branch_and_bound_MILP(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, lp_error_code):
    number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary = tf.constant(value = 0, dtype = tf.int64)
    last_variable_that_still_needs_to_become_integer_or_binary_index = tf.constant(value = 0, dtype = tf.int64)

    error_code = tf.constant(value = 0, dtype = tf.int64)

    # count_number_of_variables_needing_to_be_integer_or_binary_that_already_are
    number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index = \
        count_number_of_variables_needing_to_be_integer_or_binary_that_already_are(number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, basic_feasible_solution)

    condition_still_need_integer_or_binary_variables = tf.less(x = number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, y = number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary)
    
    def still_need_integer_or_binary_variables(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, optimal_objective_function_value, optimal_variable_values):
        # Create variables to track the best optimal values and the corresponding variables
        best_milp_error_code = tf.constant(value = 1, dtype = tf.int64)

        def maximization():
            best_mixed_integer_optimal_value_double = tf.constant(value = np.finfo(np.float64).min, dtype = tf.float64) # negative infinity
            best_mixed_integer_optimal_value = tf.constant(value = [np.iinfo(np.int64).min, 1], dtype = tf.int64)
            return best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value
        
        def minimization():
            best_mixed_integer_optimal_value_double = tf.constant(value = np.finfo(np.float64).max, dtype = tf.float64) # positive infinity
            best_mixed_integer_optimal_value = tf.constant(value = [np.iinfo(np.int64).max, 1], dtype = tf.int64)
            return best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value
        
        condition = tf.equal(x = maximization_problem, y = 1)
    
        best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value = tf.cond(pred = condition, true_fn = lambda: maximization(), false_fn = lambda: minimization(), strict = True)

        best_mixed_integer_variable_values = tf.stack(values = [tf.zeros(shape = [number_of_variables], dtype = tf.int64), tf.ones(shape = [number_of_variables], dtype = tf.int64)], axis = 0)

        # Call iterative branch and bound function
        # branch_and_bound_MILP_iterative
        best_milp_error_code, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
            branch_and_bound_MILP_iterative(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code)

        error_code = best_milp_error_code

        condition_feasible_milp = tf.equal(x = best_milp_error_code, y = 0)
        condition_infeasible_milp = tf.equal(x = best_milp_error_code, y = 1)
        condition_unbounded_milp = tf.equal(x = best_milp_error_code, y = 2)
        
        def feasible_milp(number_of_variables, best_mixed_integer_optimal_value, best_mixed_integer_variable_values):
            print("\nbranch_and_bound_MILP: still_need_integer_or_binary_variables: feasible_milp: Optimal MILP!")
            
            # Update overall optimal solution
            optimal_objective_function_value = best_mixed_integer_optimal_value[:]
            optimal_variable_values = best_mixed_integer_variable_values[:, :number_of_variables]
            
            return optimal_objective_function_value, optimal_variable_values
        
        def infeasible_milp(optimal_objective_function_value, optimal_variable_values):
            print("\nbranch_and_bound_MILP: still_need_integer_or_binary_variables: infeasible_milp: Infeasible MILP!")
            
            return optimal_objective_function_value, optimal_variable_values
        
        def unbounded_milp(optimal_objective_function_value, optimal_variable_values):
            print("\nbranch_and_bound_MILP: still_need_integer_or_binary_variables: unbounded_milp: Unbounded MILP!")
            
            return optimal_objective_function_value, optimal_variable_values
        
        def do_nothing(optimal_objective_function_value, optimal_variable_values):
            return optimal_objective_function_value, optimal_variable_values
        
        unbounded_milp_branch = tf.cond(
            pred = condition_unbounded_milp, 
            true_fn = lambda: unbounded_milp(optimal_objective_function_value, optimal_variable_values), 
            false_fn = lambda: do_nothing(optimal_objective_function_value, optimal_variable_values),
            strict = True)
        
        infeasible_milp_branch = tf.cond(
            pred = condition_infeasible_milp, 
            true_fn = lambda: infeasible_milp(optimal_objective_function_value, optimal_variable_values), 
            false_fn = lambda: unbounded_milp_branch,
            strict = True)
        
        feasible_milp_branch = tf.cond(
            pred = condition_feasible_milp, 
            true_fn = lambda: feasible_milp(number_of_variables, best_mixed_integer_optimal_value, best_mixed_integer_variable_values), 
            false_fn = lambda: infeasible_milp_branch,
            strict = True)
        
        optimal_objective_function_value, optimal_variable_values = feasible_milp_branch
        
        return error_code, optimal_objective_function_value, optimal_variable_values
            
    # If all variables that need to be integer or binary are already            
    def dont_still_need_integer_or_binary_variables(error_code, optimal_objective_function_value, optimal_variable_values, lp_error_code):
        condition_lp_optimal = tf.equal(x = lp_error_code, y = 0)
        condition_lp_unbounded = tf.equal(x = lp_error_code, y = 2)
        condition_should_not_have_gotten_into_branch_and_bound = tf.logical_not(x = tf.reduce_any(input_tensor = [condition_lp_optimal, condition_lp_unbounded]))
        
        def lp_optimal():
            error_code = tf.constant(value = 0, dtype = tf.int64)
            print("\nbranch_and_bound_MILP: dont_still_need_integer_or_binary_variables: lp_optimal: If there were any integer or binary constraints, they were already satisfied before MILP BB by optimal LP!")
            
            return error_code
        
        def lp_unbounded():
            error_code = tf.constant(value = 2, dtype = tf.int64)
            print("\nbranch_and_bound_MILP: dont_still_need_integer_or_binary_variables: lp_unbounded: LP was unbounded and since computers can't represent irrational numbers, then MILP is unbounded!")
            
            return error_code
        
        def should_not_have_gotten_into_branch_and_bound():
            error_code = tf.constant(value = 999, dtype = tf.int64)
            print("\nbranch_and_bound_MILP: dont_still_need_integer_or_binary_variables: should_not_have_gotten_into_branch_and_bound: HOW DID I GET IN MILP BB?! ERROR!")
            
            return error_code
        
        def do_nothing(error_code):
            return error_code
        
        should_not_have_gotten_into_branch_and_bound_branch = tf.cond(
            pred = condition_should_not_have_gotten_into_branch_and_bound, 
            true_fn = lambda: should_not_have_gotten_into_branch_and_bound(), 
            false_fn = lambda: do_nothing(error_code),
            strict = True)
        
        lp_unbounded_branch = tf.cond(
            pred = condition_lp_unbounded, 
            true_fn = lambda: lp_unbounded(), 
            false_fn = lambda: should_not_have_gotten_into_branch_and_bound_branch, 
            strict = True)
        
        lp_optimal_branch = tf.cond(
            pred = condition_lp_optimal, 
            true_fn = lambda: lp_optimal(), 
            false_fn = lambda: lp_unbounded_branch,
            strict = True)
        
        error_code = lp_optimal_branch
        
        return error_code, optimal_objective_function_value, optimal_variable_values
        
    error_code, optimal_objective_function_value, optimal_variable_values = tf.cond(
        pred = condition_still_need_integer_or_binary_variables, 
        true_fn = lambda: still_need_integer_or_binary_variables(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, optimal_objective_function_value, optimal_variable_values), 
        false_fn = lambda: dont_still_need_integer_or_binary_variables(error_code, optimal_objective_function_value, optimal_variable_values, lp_error_code),
        strict = True)

    return error_code, optimal_objective_function_value, optimal_variable_values

In [34]:
# This function counts the number of variables that need to be integer or binary AND already are
def count_number_of_variables_needing_to_be_integer_or_binary_that_already_are(number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, basic_feasible_solution):
    condition_variables_required_integer_or_binary = tf.greater(x = number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary, y = 0)
    
    def variables_required_integer_or_binary(number_of_variables, variable_special_requirements, basic_feasible_solution):
        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary = tf.constant(value = 0, dtype = tf.int64)
        last_variable_that_still_needs_to_become_integer_or_binary_index = tf.constant(value = -1, dtype = tf.int64)
        
        # Integer
        integer_last_variable_that_still_needs_to_become_integer_or_binary_index = tf.constant(value = -1, dtype = tf.int64)
        
        integer_variable_requirement_mask = tf.equal(x = variable_special_requirements[:], y = 1)

        basic_feasible_solution_denominator_equals_one_mask = tf.equal(x = basic_feasible_solution[1, :number_of_variables], y = 1)

        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary += tf.count_nonzero(input_tensor = tf.reduce_all(input_tensor = [integer_variable_requirement_mask, basic_feasible_solution_denominator_equals_one_mask], axis = 0))

        integer_still_needs_all_array = tf.reduce_all(input_tensor = [integer_variable_requirement_mask, tf.logical_not(x = basic_feasible_solution_denominator_equals_one_mask)], axis = 0)
        
        integer_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = number_of_variables, dtype = tf.int64), mask = integer_still_needs_all_array)
        
        integer_last_variable_that_still_needs_to_become_integer_or_binary_index = tf.reduce_max(input_tensor = tf.concat(values = [integer_indices, -tf.ones(shape = [1], dtype = tf.int64)], axis = 0))
        
        # Binary
        binary_last_variable_that_still_needs_to_become_integer_or_binary_index = tf.constant(value = -1, dtype = tf.int64)
    
        binary_variable_requirement_mask = tf.equal(x = variable_special_requirements[:], y = 2)

        basic_feasible_solution_numerator_equals_zero_mask = tf.equal(x = basic_feasible_solution[0, :number_of_variables], y = 0)

        basic_feasible_solution_numerator_equals_denominator_mask = tf.equal(x = basic_feasible_solution[0, :number_of_variables], y = basic_feasible_solution[1, :number_of_variables])

        count_any_array = tf.reduce_any(input_tensor = [basic_feasible_solution_numerator_equals_zero_mask, basic_feasible_solution_numerator_equals_denominator_mask], axis = 0)
        
        count_all_array = tf.reduce_all(input_tensor = [binary_variable_requirement_mask, count_any_array], axis = 0)

        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary += tf.count_nonzero(input_tensor = count_all_array)

        basic_feasible_solution_float_between_zero_and_one_mask = tf.reduce_all(input_tensor = [tf.greater_equal(x = tf.cast(x = basic_feasible_solution[0, :number_of_variables], dtype = tf.float64) / tf.cast(x = basic_feasible_solution[1, :number_of_variables], dtype = tf.float64), y = 0), tf.less_equal(x = tf.cast(x = basic_feasible_solution[0, :number_of_variables], dtype = tf.float64) / tf.cast(x = basic_feasible_solution[1, :number_of_variables], dtype = tf.float64), y = 1)], axis = 0)

        binary_still_needs_all_array = tf.reduce_all(input_tensor = [binary_variable_requirement_mask, tf.logical_not(x = basic_feasible_solution_numerator_equals_zero_mask), tf.logical_not(x = basic_feasible_solution_numerator_equals_denominator_mask), basic_feasible_solution_float_between_zero_and_one_mask], axis = 0)
        
        binary_indices = tf.boolean_mask(tensor = tf.range(start = 0, limit = number_of_variables, dtype = tf.int64), mask = binary_still_needs_all_array)
        
        binary_last_variable_that_still_needs_to_become_integer_or_binary_index = tf.reduce_max(input_tensor = tf.concat(values = [binary_indices, -tf.ones(shape = [1], dtype = tf.int64)], axis = 0))

        last_variable_that_still_needs_to_become_integer_or_binary_index = tf.reduce_max(input_tensor = [integer_last_variable_that_still_needs_to_become_integer_or_binary_index, binary_last_variable_that_still_needs_to_become_integer_or_binary_index])
        
        return number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index
    
    def no_variables_required_integer_or_binary():
        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary = tf.constant(value = 0, dtype = tf.int64)
        last_variable_that_still_needs_to_become_integer_or_binary_index = tf.constant(value = -1, dtype = tf.int64)
        
        return number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index
    
    number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index = \
        tf.cond(
            pred = condition_variables_required_integer_or_binary, 
            true_fn = lambda: variables_required_integer_or_binary(number_of_variables, variable_special_requirements, basic_feasible_solution), 
            false_fn = lambda: no_variables_required_integer_or_binary(),
            strict = True)
    
    return number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index

In [35]:
# This function recursively applies branch and bound
def branch_and_bound_MILP_iterative(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code):
    # Create add constraint stack
    constraint_stack_count = tf.constant(value = 0, dtype = tf.int64)
    
    branch_and_bound_add_constraint_stack_variable_index = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_add_constraint_stack_constraint_inequality_direction = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_add_constraint_stack_constraint_constant = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_add_constraint_stack_recursion_level = tf.zeros(shape = [0], dtype = tf.int64)
    
    # Create state stack
    recursion_level = tf.constant(value = 0, dtype = tf.int64)
    
    branch_and_bound_state_stack_tableau_current_size = tf.zeros(shape = [0, 2], dtype = tf.int64)
    branch_and_bound_state_stack_number_of_constraints = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_state_stack_number_of_slack_surplus_variables = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_state_stack_number_of_artificial_variables = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_state_stack_basic_variables = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_state_stack_basic_feasible_solution = tf.zeros(shape = [2, 0], dtype = tf.int64)
    branch_and_bound_state_stack_tableau_matrix = tf.zeros(shape = [2, 0], dtype = tf.int64)
    branch_and_bound_state_stack_basic_variables_index_offsets = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_state_stack_basic_feasible_solution_index_offsets = tf.zeros(shape = [0], dtype = tf.int64)
    branch_and_bound_state_stack_tableau_matrix_index_offsets = tf.zeros(shape = [1], dtype = tf.int64)
    
    # This will be the value that we try next for the corresponding variable
    last_variable_that_still_needs_to_become_integer_value = tf.cast(x = basic_feasible_solution[0, last_variable_that_still_needs_to_become_integer_or_binary_index], dtype = tf.float64) / tf.cast(x = basic_feasible_solution[1, last_variable_that_still_needs_to_become_integer_or_binary_index], dtype = tf.float64)

    # Push onto add constraint stack
    branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count = \
        push_branch_and_bound_add_constraint_to_stack(last_variable_that_still_needs_to_become_integer_or_binary_index, last_variable_that_still_needs_to_become_integer_value, recursion_level, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level)

    # Push onto state stack
    branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
        push_branch_and_bound_state_to_stack(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)
    
    def while_loop_condition(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        condition = tf.greater(x = constraint_stack_count, y = 0)
        
        return condition

    def while_loop_body(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        # Pop off top constraint to add from constraint stack
        new_constraint_variable_index, new_constraint_constraint_inequality_direction, new_constraint_constraint_constant, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count = \
            pop_new_constraint_from_branch_and_bound_add_constraint_stack(constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level)
        
        # Evaluate new constraint
        number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets = \
            branch_and_bound_MILP_iterative_less_or_greater_than(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, new_constraint_variable_index, new_constraint_constraint_inequality_direction, new_constraint_constraint_constant, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)

        return maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

    maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets = \
        tf.while_loop(
            cond = while_loop_condition, 
            body = while_loop_body, 
            loop_vars = [maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets],
            shape_invariants = [maximization_problem.get_shape(), number_of_constraints.get_shape(), number_of_slack_surplus_variables.get_shape(), number_of_artificial_variables.get_shape(), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [2, None]), tf.TensorShape(dims = [2, None, None]), tableau_current_size.get_shape(), best_milp_error_code.get_shape(), best_mixed_integer_optimal_value_double.get_shape(), best_mixed_integer_optimal_value.get_shape(), tf.TensorShape(dims = [2, None]), constraint_stack_count.get_shape(), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]), recursion_level.get_shape(), tf.TensorShape(dims = [None, 2]),tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]),tf.TensorShape(dims = [2, None]), tf.TensorShape(dims = [2, None]), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None]), tf.TensorShape(dims = [None])])
    
    return best_milp_error_code, best_mixed_integer_optimal_value, best_mixed_integer_variable_values

In [36]:
# This function pushes constraints to add in branch and bound to stack
def push_branch_and_bound_add_constraint_to_stack(last_variable_that_still_needs_to_become_integer_or_binary_index, last_variable_that_still_needs_to_become_integer_value, recursion_level, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level):
    # Push greater than
    branch_and_bound_add_constraint_stack_variable_index = tf.concat(values = [branch_and_bound_add_constraint_stack_variable_index, tf.expand_dims(input = last_variable_that_still_needs_to_become_integer_or_binary_index, axis = -1)], axis = 0)
    branch_and_bound_add_constraint_stack_constraint_inequality_direction = tf.concat(values = [branch_and_bound_add_constraint_stack_constraint_inequality_direction, tf.expand_dims(input = tf.constant(value = -1, dtype = tf.int64), axis = -1)], axis = 0)
    branch_and_bound_add_constraint_stack_constraint_constant = tf.concat(values = [branch_and_bound_add_constraint_stack_constraint_constant, tf.expand_dims(input = tf.cast(x = tf.ceil(x = last_variable_that_still_needs_to_become_integer_value), dtype = tf.int64), axis = -1)], axis = 0)
    branch_and_bound_add_constraint_stack_recursion_level = tf.concat(values = [branch_and_bound_add_constraint_stack_recursion_level, tf.expand_dims(input = recursion_level, axis = -1)], axis = 0)

    constraint_stack_count += 1
    
    # Push less than
    branch_and_bound_add_constraint_stack_variable_index = tf.concat(values = [branch_and_bound_add_constraint_stack_variable_index, tf.expand_dims(input = last_variable_that_still_needs_to_become_integer_or_binary_index, axis = -1)], axis = 0)
    branch_and_bound_add_constraint_stack_constraint_inequality_direction = tf.concat(values = [branch_and_bound_add_constraint_stack_constraint_inequality_direction, tf.expand_dims(input = tf.constant(value = 1, dtype = tf.int64), axis = -1)], axis = 0)
    branch_and_bound_add_constraint_stack_constraint_constant = tf.concat(values = [branch_and_bound_add_constraint_stack_constraint_constant, tf.expand_dims(input = tf.cast(x = tf.floor(x = last_variable_that_still_needs_to_become_integer_value), dtype = tf.int64), axis = -1)], axis = 0)
    branch_and_bound_add_constraint_stack_recursion_level = tf.concat(values = [branch_and_bound_add_constraint_stack_recursion_level, tf.expand_dims(input = recursion_level, axis = -1)], axis = 0)
    
    constraint_stack_count += 1
    
    return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count

In [37]:
# This function pops a new constraint to add in branch and bound from stack
def pop_new_constraint_from_branch_and_bound_add_constraint_stack(constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level):
    constraint_stack_count -= 1

    new_constraint_variable_index = branch_and_bound_add_constraint_stack_variable_index[-1]
    branch_and_bound_add_constraint_stack_variable_index = branch_and_bound_add_constraint_stack_variable_index[0:-1]

    new_constraint_constraint_inequality_direction = branch_and_bound_add_constraint_stack_constraint_inequality_direction[-1]
    branch_and_bound_add_constraint_stack_constraint_inequality_direction = branch_and_bound_add_constraint_stack_constraint_inequality_direction[0:-1]

    new_constraint_constraint_constant = branch_and_bound_add_constraint_stack_constraint_constant[-1]
    branch_and_bound_add_constraint_stack_constraint_constant = branch_and_bound_add_constraint_stack_constraint_constant[0:-1]

    new_constraint_recursion_level = branch_and_bound_add_constraint_stack_recursion_level[-1]
    branch_and_bound_add_constraint_stack_recursion_level = branch_and_bound_add_constraint_stack_recursion_level[0:-1]

    return new_constraint_variable_index, new_constraint_constraint_inequality_direction, new_constraint_constraint_constant, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count

In [38]:
# This function pushes branch and bound state to the stack
def push_branch_and_bound_state_to_stack(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
    condition = tf.less(x = recursion_level, y = max_branch_and_bound_recursion_depth + 1)

    def add_to_stack(branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level):
        reshaped_tableau_matrix = tf.reshape(tensor = tableau_matrix[:, 0:tableau_current_size[0], 0:tableau_current_size[1]], shape = [2, tableau_current_size[0] * tableau_current_size[1]])

        branch_and_bound_state_stack_tableau_current_size = tf.concat(values = [branch_and_bound_state_stack_tableau_current_size, tf.expand_dims(input = tableau_current_size, axis = 0)], axis = 0)
        branch_and_bound_state_stack_number_of_constraints = tf.concat(values = [branch_and_bound_state_stack_number_of_constraints, tf.expand_dims(input = number_of_constraints, axis = -1)], axis = 0)
        branch_and_bound_state_stack_number_of_slack_surplus_variables = tf.concat(values = [branch_and_bound_state_stack_number_of_slack_surplus_variables, tf.expand_dims(input = number_of_slack_surplus_variables, axis = -1)], axis = 0)
        branch_and_bound_state_stack_number_of_artificial_variables = tf.concat(values = [branch_and_bound_state_stack_number_of_artificial_variables, tf.expand_dims(input = number_of_artificial_variables, axis = -1)], axis = 0)

        branch_and_bound_state_stack_basic_variables = tf.concat(values = [branch_and_bound_state_stack_basic_variables, basic_variables[0:number_of_constraints]], axis = 0)
        branch_and_bound_state_stack_basic_feasible_solution = tf.concat(values = [branch_and_bound_state_stack_basic_feasible_solution, basic_feasible_solution[:, 0:number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables]], axis = 1)
        branch_and_bound_state_stack_tableau_matrix = tf.concat(values = [branch_and_bound_state_stack_tableau_matrix, reshaped_tableau_matrix], axis = 1)

        branch_and_bound_state_stack_basic_variables_index_offsets = tf.cumsum(x = tf.concat(values = [tf.expand_dims(input = tf.constant(value = 0, dtype = tf.int64), axis = -1), branch_and_bound_state_stack_number_of_constraints], axis = 0))
        branch_and_bound_state_stack_basic_feasible_solution_index_offsets = tf.cumsum(x = tf.concat(values = [tf.expand_dims(input = tf.constant(value = 0, dtype = tf.int64), axis = -1), number_of_variables + branch_and_bound_state_stack_number_of_slack_surplus_variables + branch_and_bound_state_stack_number_of_artificial_variables], axis = 0))
        branch_and_bound_state_stack_tableau_matrix_index_offsets = tf.concat(values = [branch_and_bound_state_stack_tableau_matrix_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets[-1] + tf.expand_dims(input = tableau_current_size[0] * tableau_current_size[1], axis = 0)], axis = 0)

        return branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

    def stack_overflow(branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level):
        print("push_branch_and_bound_state_to_stack: stack_overflow: Stack overflow!")

        return branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

    branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
        tf.cond(
            pred = condition, 
            true_fn = lambda: add_to_stack(branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level), 
            false_fn = lambda: stack_overflow(branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level), 
            strict = True)
    
    return branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

In [39]:
# This function deletes a branch and bound state from the stack
def delete_branch_and_bound_state_from_stack(recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
    branch_and_bound_state_stack_tableau_current_size = branch_and_bound_state_stack_tableau_current_size[0:-1, :]

    branch_and_bound_state_stack_number_of_constraints = branch_and_bound_state_stack_number_of_constraints[0:-1]

    branch_and_bound_state_stack_number_of_slack_surplus_variables = branch_and_bound_state_stack_number_of_slack_surplus_variables[0:-1]

    branch_and_bound_state_stack_number_of_artificial_variables = branch_and_bound_state_stack_number_of_artificial_variables[0:-1]


    branch_and_bound_state_stack_basic_variables = tf.gather(params = branch_and_bound_state_stack_basic_variables, indices = tf.range(start = 0, limit = branch_and_bound_state_stack_basic_variables_index_offsets[-2], dtype = tf.int64))

    branch_and_bound_state_stack_basic_feasible_solution = tf.gather(params = branch_and_bound_state_stack_basic_feasible_solution, indices = tf.range(start = 0, limit = branch_and_bound_state_stack_basic_feasible_solution_index_offsets[-2], dtype = tf.int64), axis = 1)

    branch_and_bound_state_stack_tableau_matrix = tf.gather(params = branch_and_bound_state_stack_tableau_matrix, indices = tf.range(start = 0, limit = branch_and_bound_state_stack_tableau_matrix_index_offsets[-2], dtype = tf.int64), axis = 1)

    branch_and_bound_state_stack_basic_variables_index_offsets = branch_and_bound_state_stack_basic_variables_index_offsets[0:-1]
    branch_and_bound_state_stack_basic_feasible_solution_index_offsets = branch_and_bound_state_stack_basic_feasible_solution_index_offsets[0:-1]
    branch_and_bound_state_stack_tableau_matrix_index_offsets = branch_and_bound_state_stack_tableau_matrix_index_offsets[0:-1]
        
    return branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

In [40]:
# This function handles the less than or greater than branches of the recursive tree of branch and bound
def branch_and_bound_MILP_iterative_less_or_greater_than(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, new_constraint_variable_index, new_constraint_constraint_inequality_direction, new_constraint_constraint_constant, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
    condition_new_constraint_constraint_inequality_direction = tf.equal(x = new_constraint_constraint_inequality_direction, y = 1)

    # add_constraint
    error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size = \
        add_constraint(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, new_constraint_variable_index, new_constraint_constraint_inequality_direction, new_constraint_constraint_constant, recursion_level)

    condition_milp_feasible = tf.equal(x = error_code, y = 0)
    condition_milp_infeasible = tf.equal(x = error_code, y = 1)
    condition_milp_unbounded = tf.equal(x = error_code, y = 2)
    
    def milp_feasible(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        # count_number_of_variables_needing_to_be_integer_or_binary_that_already_are
        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive = \
            count_number_of_variables_needing_to_be_integer_or_binary_that_already_are(number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, basic_feasible_solution)

        condition_still_need_integer_or_binary_varibles = tf.less(x = number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, y = number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary)

        def check_if_more(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            # check_if_more_branch_and_bound_MILP_recursion_is_necessary
            branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
                check_if_more_branch_and_bound_MILP_recursion_is_necessary(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)

            return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

        def save_best(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            print_optimal_results(tableau_matrix[:, number_of_constraints, 0], basic_feasible_solution)

            # save_best_mixed_integer_optimal_variables_and_values
            best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
                save_best_mixed_integer_optimal_variables_and_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code)

            return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

        best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets = \
            tf.cond(
                pred = condition_still_need_integer_or_binary_varibles, 
                true_fn = lambda: check_if_more(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
                false_fn = lambda: save_best(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets),
                strict = True)
        
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

    def milp_infeasible(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

    def milp_unbounded(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        condition_best_milp_error_code_infeasible = tf.equal(x = best_milp_error_code, y = 1)

        def best_milp_error_code_infeasible():
            best_milp_error_code = tf.constant(value = 2, dtype = tf.int64)

            return best_milp_error_code

        def best_milp_error_code_not_infeasible(best_milp_error_code):
            return best_milp_error_code

        best_milp_error_code = tf.cond(
            pred = condition_best_milp_error_code_infeasible, 
            true_fn = lambda: best_milp_error_code_infeasible(), 
            false_fn = lambda: best_milp_error_code_not_infeasible(best_milp_error_code),
            strict = True)

        print("branch_and_bound_MILP_iterative_less_or_greater_than: milp_unbounded: Unbounded!")
        
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

    def do_nothing(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

    def milp_unbounded_branch_function(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        milp_unbounded_branch = tf.cond(
            pred = condition_milp_unbounded, 
            true_fn = lambda: milp_unbounded(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            false_fn = lambda: do_nothing(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            strict = True)
        
        return milp_unbounded_branch

    def milp_infeasible_branch_function(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        milp_infeasible_branch = tf.cond(
            pred = condition_milp_infeasible, 
            true_fn = lambda: milp_infeasible(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            false_fn = lambda: milp_unbounded_branch_function(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            strict = True)
        
        return milp_infeasible_branch

    def milp_feasible_branch_function(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        milp_feasible_branch = tf.cond(
            pred = condition_milp_feasible, 
            true_fn = lambda: milp_feasible(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            false_fn = lambda: milp_infeasible_branch_function(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            strict = True)
        
        return milp_feasible_branch
    
    best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets = \
        milp_feasible_branch_function(maximization_problem, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)
    
    # Pop state off from state stack to delete
    def reset_less_than(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix = \
            reset_branch_and_bound_MILP_recursive_counts_and_arrays(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)
        
        return tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

    def reset_greater_than(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        condition = tf.greater(x = recursion_level, y = 0)

        def reset_counts_and_arrays(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
                delete_branch_and_bound_state_from_stack(recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)
            
            tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix = reset_branch_and_bound_MILP_recursive_counts_and_arrays(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level - 1, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)
            
            return tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

        def do_nothing(tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
                delete_branch_and_bound_state_from_stack(recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)
            
            return tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

        tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets = \
            tf.cond(
                pred = condition, 
                true_fn = lambda: reset_counts_and_arrays(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
                false_fn = lambda: do_nothing(tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets),
                strict = True)
        
        recursion_level -= 1

        return tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

    tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
        tf.cond(
            pred = condition_new_constraint_constraint_inequality_direction, 
            true_fn = lambda: reset_less_than(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
            false_fn = lambda: reset_greater_than(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets),
            strict = True)

    return number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets

In [41]:
# This function checks if more recursion of MILP branch and bound is necessary
def check_if_more_branch_and_bound_MILP_recursion_is_necessary(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
    condition_not_at_max_recursion_depth = tf.less(x = recursion_level, y = max_branch_and_bound_recursion_depth)
    
    def continue_checking(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
        def continue_down_rabbit_hole(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            recursion_level += 1

            # This will be the value that we try next for the corresponding variable
            last_variable_that_still_needs_to_become_integer_value = tf.cast(x = basic_feasible_solution[0, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive], dtype = tf.float64) / tf.cast(x = basic_feasible_solution[1, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive], dtype = tf.float64)

            # Push onto add constraint stack
            branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count = \
                push_branch_and_bound_add_constraint_to_stack(last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, last_variable_that_still_needs_to_become_integer_value, recursion_level, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level)

            # Push onto state stack
            branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
                push_branch_and_bound_state_to_stack(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets)

            return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level
        
        def dont_continue_down_rabbit_hole(branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level):
            return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level
    
        condition_maximization = tf.equal(x = maximization_problem, y = 1)
        
        def maximization(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            condition_better_objective_value = tf.greater(x = tf.cast(x = tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64), y = best_mixed_integer_optimal_value_double)
            condition_more_depth_left = tf.less(x = recursion_level, y = max_branch_and_bound_recursion_depth)
            condition = tf.logical_and(x = condition_better_objective_value, y = condition_more_depth_left)

            branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
                tf.cond(
                    pred = condition, 
                    true_fn = lambda: continue_down_rabbit_hole(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
                    false_fn = lambda: dont_continue_down_rabbit_hole(branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level),
                    strict = True)

            return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

        def minimization(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
            condition_better_objective_value = tf.less(x = tf.cast(x = -tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64), y = best_mixed_integer_optimal_value_double)
            condition_more_depth_left = tf.less(x = recursion_level, y = max_branch_and_bound_recursion_depth)
            condition = tf.logical_and(x = condition_better_objective_value, y = condition_more_depth_left)

            branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
                tf.cond(
                    pred = condition, 
                    true_fn = lambda: continue_down_rabbit_hole(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
                    false_fn = lambda: dont_continue_down_rabbit_hole(branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level),
                    strict = True)

            return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level
        
        branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
            tf.cond(
                pred = condition_maximization, 
                true_fn = lambda: maximization(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets), 
                false_fn = lambda: minimization(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets),
                strict = True)
        
        return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level
    
    branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level = \
        tf.cond(
            pred = condition_not_at_max_recursion_depth,
            true_fn = lambda: continue_checking(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_mixed_integer_optimal_value_double, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, constraint_stack_count, branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets),
            false_fn = lambda: (branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level),
            strict = True)

    return branch_and_bound_add_constraint_stack_variable_index, branch_and_bound_add_constraint_stack_constraint_inequality_direction, branch_and_bound_add_constraint_stack_constraint_constant, branch_and_bound_add_constraint_stack_recursion_level, constraint_stack_count, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets, recursion_level

In [42]:
# This function saves the best mixed integer optimal variables and values
def save_best_mixed_integer_optimal_variables_and_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code):
    def save_best_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code):
        best_milp_error_code = tf.constant(value = 0, dtype = tf.int64)

        best_mixed_integer_optimal_value_double = tf.cast(x = tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64)

        best_mixed_integer_optimal_value = tableau_matrix[:, number_of_constraints, 0]

        best_mixed_integer_variable_values = basic_feasible_solution[:, 0:number_of_variables]
        
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values
    
    def dont_save_best_values(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values):
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values
            
    def maximization(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code):
        condition = tf.greater(x = tf.cast(x = tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64), y = best_mixed_integer_optimal_value_double)
        best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
            tf.cond(
                pred = condition, 
                true_fn = lambda: save_best_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code), 
                false_fn = lambda: dont_save_best_values(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values))
        
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values
    
    def minimization(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code):
        condition = tf.less(x = tf.cast(x = tableau_matrix[0, number_of_constraints, 0], dtype = tf.float64) / tf.cast(x = tableau_matrix[1, number_of_constraints, 0], dtype = tf.float64), y = best_mixed_integer_optimal_value_double)
        best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
            tf.cond(
                pred = condition, 
                true_fn = lambda: save_best_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code), 
                false_fn = lambda: dont_save_best_values(best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values))
        
        return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values
            
    condition = tf.equal(x = maximization_problem, y = 1)
    
    best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
        tf.cond(
            pred = condition, 
            true_fn = lambda: maximization(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code), 
            false_fn = lambda: minimization(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tf.stack(values = [-tableau_matrix[0], tableau_matrix[1]], axis = 0), best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code))
                
    return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values

In [43]:
# This function resets the counts and arrays during recursive branch and bound MILP
def reset_branch_and_bound_MILP_recursive_counts_and_arrays(number_of_variables, tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, recursion_level, branch_and_bound_state_stack_tableau_current_size, branch_and_bound_state_stack_number_of_constraints, branch_and_bound_state_stack_number_of_slack_surplus_variables, branch_and_bound_state_stack_number_of_artificial_variables, branch_and_bound_state_stack_basic_variables, branch_and_bound_state_stack_basic_feasible_solution, branch_and_bound_state_stack_tableau_matrix, branch_and_bound_state_stack_basic_variables_index_offsets, branch_and_bound_state_stack_basic_feasible_solution_index_offsets, branch_and_bound_state_stack_tableau_matrix_index_offsets):
    # Reset counts
    tableau_current_size = branch_and_bound_state_stack_tableau_current_size[-1]

    number_of_constraints = branch_and_bound_state_stack_number_of_constraints[-1]

    number_of_slack_surplus_variables = branch_and_bound_state_stack_number_of_slack_surplus_variables[-1]

    number_of_artificial_variables = branch_and_bound_state_stack_number_of_artificial_variables[-1]

    # Reset arrays
    basic_variables = tf.gather(params = branch_and_bound_state_stack_basic_variables, indices = tf.range(start = branch_and_bound_state_stack_basic_variables_index_offsets[-2], limit = branch_and_bound_state_stack_basic_variables_index_offsets[-1], dtype = tf.int64))

    basic_feasible_solution = tf.gather(params = branch_and_bound_state_stack_basic_feasible_solution, indices = tf.range(start = branch_and_bound_state_stack_basic_feasible_solution_index_offsets[-2], limit = branch_and_bound_state_stack_basic_feasible_solution_index_offsets[-1], dtype = tf.int64), axis = 1)

    tableau_matrix = tf.gather(params = branch_and_bound_state_stack_tableau_matrix, indices = tf.range(start = branch_and_bound_state_stack_tableau_matrix_index_offsets[-2], limit = branch_and_bound_state_stack_tableau_matrix_index_offsets[-1], dtype = tf.int64), axis = 1)
    tableau_matrix = tf.reshape(tensor = tableau_matrix, shape = tf.concat(values = [tf.expand_dims(input = tf.constant(value = 2, dtype = tf.int64), axis = -1), tableau_current_size], axis = 0))
    tableau_matrix = tf.ensure_shape(x = tableau_matrix, shape = [2, None, None])

    return tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix

In [44]:
# This function adds a constraint to the previous optimal solution (warm-start)
def add_constraint(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, last_variable_that_still_needs_to_become_integer_or_binary_index, new_constraint_inequality_direction, new_constraint_constant, recursion_level):
    # Increase tableau size to make room for two new rows and columns due to the new constraint
    tableau_current_size += 2

    # Take contiguous slice from incoming tableau that will be carried over to new tableau
    original_tableau_matrix_slice = tableau_matrix[:, 0:tableau_current_size[0] - 3, 0:tableau_current_size[1] - 2]

    # Take the incoming tableau's objective function row
    original_tableau_matrix_objective_row = tf.expand_dims(input = tableau_matrix[:, tableau_current_size[0] - 3, 0:tableau_current_size[1] - 2], axis = 1)

    # Create new artificial objective row for phase 1 simplex
    new_artificial_objective_row = tf.stack(values = [tf.zeros(shape = [1, tableau_current_size[1] - 2], dtype = tf.int64), tf.ones(shape = [1, tableau_current_size[1] - 2], dtype = tf.int64)], axis = 0)

    # Create new constraint row
    new_constraint_constant_element = tf.stack(values = [tf.reshape(tensor = -new_constraint_inequality_direction * new_constraint_constant, shape = [1, 1]), tf.ones(shape = [1, 1], dtype = tf.int64)], axis = 0)

    variable_mask = tf.equal(x = tf.range(start = 0, limit = number_of_variables, dtype = tf.int64), y = last_variable_that_still_needs_to_become_integer_or_binary_index)

    new_constraint_variable_vector = tf.stack(values = [tf.expand_dims(input = tf.where(condition = variable_mask, x = -new_constraint_inequality_direction * tf.ones(shape = [number_of_variables], dtype = tf.int64), y = tf.zeros(shape = [number_of_variables], dtype = tf.int64)), axis = 0), tf.ones(shape = [1, number_of_variables], dtype = tf.int64)], axis = 0)

    old_constraints_slack_surplus_zero_vector = tf.stack(values = [tf.zeros(shape = [1, number_of_constraints], dtype = tf.int64), tf.ones(shape = [1, number_of_constraints], dtype = tf.int64)], axis = 0)

    new_constraint_row = tf.concat(values = [new_constraint_constant_element, new_constraint_variable_vector, old_constraints_slack_surplus_zero_vector], axis = 2)
    
    basic_variable_mask = tf.equal(x = basic_variables[0:number_of_constraints], y = last_variable_that_still_needs_to_become_integer_or_binary_index + 1)
    
    basic_variable_index = tf.squeeze(input = tf.boolean_mask(tensor = tf.range(start = 0, limit = number_of_constraints, dtype = tf.int64), mask = basic_variable_mask))

    # Add variable's basic row to new constraint row, if less than or equal to
    true_array = tf.stack(values = tf.map_fn(fn = lambda j: rational_addition(new_constraint_row[0, 0, j], new_constraint_row[1, 0, j], original_tableau_matrix_slice[0, basic_variable_index, j], original_tableau_matrix_slice[1, basic_variable_index, j]), elems = tf.range(start = 0, limit = tableau_current_size[1] - 2, dtype = tf.int64), dtype = (tf.int64, tf.int64)), axis = 0)

    # Subtract variable's basic row from new constraint row, if greater than or equal to
    false_array = tf.stack(values = tf.map_fn(fn = lambda j: rational_addition(new_constraint_row[0, 0, j], new_constraint_row[1, 0, j], -original_tableau_matrix_slice[0, basic_variable_index, j], original_tableau_matrix_slice[1, basic_variable_index, j]), elems = tf.range(start = 0, limit = tableau_current_size[1] - 2, dtype = tf.int64), dtype = (tf.int64, tf.int64)), axis = 0)
    
    condition_new_constraint_row_added_or_subtracted = tf.equal(x = new_constraint_inequality_direction, y = 1)

    new_constraint_row_added_or_subtracted = tf.expand_dims(input = tf.where(condition = condition_new_constraint_row_added_or_subtracted, x = true_array, y = false_array), axis = 1)

    # This is the full new tableau minus the last two columns, in other words tableau_matrix[:, 0:tableau_current_size[0], 0:tableau_current_size[1] - 2]
    tableau_old_columns = tf.concat(values = [original_tableau_matrix_slice, new_constraint_row_added_or_subtracted, original_tableau_matrix_objective_row, new_artificial_objective_row], axis = 1)

    # Fill in last two new columns for constraint's new surplus and artificial variables
    new_surplus_and_artificial_variable_columns_numerator = tf.concat(values = [tf.zeros(shape = [number_of_constraints, 2], dtype = tf.int64), tf.constant(value = [[-1, 1]], dtype = tf.int64), tf.zeros(shape = [1, 2], dtype = tf.int64), tf.constant(value = [[0, -1]], dtype = tf.int64)], axis = 0)

    new_surplus_and_artificial_variable_columns_denominator = tf.ones(shape = [tableau_current_size[0], 2], dtype = tf.int64)

    new_surplus_and_artificial_variable_columns = tf.stack(values = [new_surplus_and_artificial_variable_columns_numerator, new_surplus_and_artificial_variable_columns_denominator], axis = 0)

    # Combine everything into the final new tableau matrix
    tableau_matrix = tf.concat(values = [tableau_old_columns, new_surplus_and_artificial_variable_columns], axis = 2)

    number_of_artificial_variables += 1

    # Update basic variables
    basic_variables = tf.concat(values = [basic_variables[0:number_of_constraints], [tableau_current_size[1] - 1]], axis = 0)

    number_of_constraints += 1
    number_of_slack_surplus_variables += 1

    # Update the basic feasible solution
    basic_feasible_solution = realloc_basic_feasible_solution(basic_feasible_solution, number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables)
    
    basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix)

    # This function finds the optimal solution for the given variables and constraints
    # simplex_algorithm
    error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = simplex_algorithm(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size)
    
    return error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size

# Helper functions

In [45]:
# This function prints the optimal objective function value and the variable values
def print_optimal_results(optimal_objective_function_value, optimal_variable_values):
    print("print_optimal_results: optimal_objective_function_value = \n{}".format(optimal_objective_function_value))
    print("print_optimal_results: optimal_objective_function_value(float) = \n{}".format(tf.cast(x = optimal_objective_function_value[0], dtype = tf.float64) / tf.cast(x = optimal_objective_function_value[1], dtype = tf.float64)))
    
    print("print_optimal_results: optimal_variable_values = \n{}".format(optimal_variable_values))
    print("print_optimal_results: optimal_variable_values(float) = \n{}".format(tf.cast(x = optimal_variable_values[0], dtype = tf.float64) / tf.cast(x = optimal_variable_values[1], dtype = tf.float64)))
        
    return

In [46]:
# This function adds two columns to basic feasible solution array
def realloc_basic_feasible_solution(basic_feasible_solution, number_of_total_variables):
    additional_basic_feasible_solution_cols = tf.stack(values = [tf.zeros(shape = [2], dtype = tf.int64), tf.ones(shape = [2], dtype = tf.int64)])

    basic_feasible_solution = tf.concat(values = [basic_feasible_solution[:,:number_of_total_variables], additional_basic_feasible_solution_cols], axis = 1)
    
    return basic_feasible_solution

In [47]:
# This function adds long long rationals and keeps their numerators and denominators each as small as possible
def rational_addition(A_numerator, A_denominator, B_numerator, B_denominator):
    C_numerator = A_numerator * B_denominator + A_denominator * B_numerator
    C_denominator = A_denominator * B_denominator
    
    temp_gcd = greastest_common_denominator(C_numerator, C_denominator)
    C_numerator = C_numerator // temp_gcd
    C_denominator = C_denominator // temp_gcd
    
    return C_numerator, C_denominator

In [48]:
# This function divides long long rationals and keeps their numerators and denominators each as small as possible
def rational_division(A_numerator, A_denominator, B_numerator, B_denominator):
    C_numerator = A_numerator * B_denominator
    C_denominator = A_denominator * B_numerator
    
    temp_gcd = greastest_common_denominator(C_numerator, C_denominator)
    C_numerator = C_numerator // temp_gcd
    C_denominator = C_denominator // temp_gcd
    
    C_numerator = tf.cond(pred = tf.less(x = C_denominator, y = 0), true_fn = lambda: -C_numerator, false_fn = lambda: C_numerator)
    C_denominator = tf.cond(pred = tf.less(x = C_denominator, y = 0), true_fn = lambda: -C_denominator, false_fn = lambda: C_denominator)
    
    return C_numerator, C_denominator

In [49]:
# This function finds the greatest common denominator between a and b
def greastest_common_denominator(a, b):
    def while_loop_condition(a, b):
        return tf.not_equal(x = b, y = 0)
    
    def while_loop_body(a, b):
        t = b
        b = a % b
        a = t
        return a, b
    
    # Call tf.while_loop
    a, b = tf.while_loop(cond = while_loop_condition, body = while_loop_body, loop_vars = [a, b])
    
    return tf.abs(x = a)

# MAIN

In [50]:
# This is the main function
def main_function():
    error_code = tf.constant(value = 0, dtype = tf.int64)

    #*******************************************************************************
    #******************************** READ INPUTS **********************************
    #*******************************************************************************

    # Optimization problem type
    maximization_problem_int = read_objective_function_maximization_or_minimization()
    
    maximization_problem = tf.constant(value = maximization_problem_int, dtype = tf.int64)
    
    # Sizes
    number_of_constraints_int, number_of_variables_int = read_number_of_constraints_and_variables()
    
    number_of_constraints = tf.constant(value = number_of_constraints_int, dtype = tf.int64)
    number_of_variables = tf.constant(value = number_of_variables_int, dtype = tf.int64)

    # Variable special requirements
    variable_special_requirements_array, number_of_variables_required_to_be_standard_int, number_of_variables_required_to_be_integer_int, number_of_variables_required_to_be_binary_int, number_of_variables_required_to_be_unrestricted_int = \
        read_and_count_variable_special_requirements()
    
    variable_special_requirements = tf.constant(value = variable_special_requirements_array, dtype = tf.int64)
    number_of_variables_required_to_be_standard = tf.constant(value = number_of_variables_required_to_be_standard_int, dtype = tf.int64)
    number_of_variables_required_to_be_integer = tf.constant(value = number_of_variables_required_to_be_integer_int, dtype = tf.int64)
    number_of_variables_required_to_be_binary = tf.constant(value = number_of_variables_required_to_be_binary_int, dtype = tf.int64)
    number_of_variables_required_to_be_unrestricted = tf.constant(value = number_of_variables_required_to_be_unrestricted_int, dtype = tf.int64)
    

    # Objective function
    objective_function_initial_constant_array = read_objective_function_initial_constant()
    
    objective_function_initial_constant = tf.constant(value = objective_function_initial_constant_array, dtype = tf.int64)

    objective_function_coefficient_vector_array = read_objective_function_coefficient_vector()
    
    objective_function_coefficient_vector = tf.constant(value = objective_function_coefficient_vector_array, dtype = tf.int64)

    # Constraints
    constraint_inequality_direction_vector_array = read_constraint_inequality_direction_vector()
    
    constraint_inequality_direction_vector = tf.constant(value = constraint_inequality_direction_vector_array, dtype = tf.int64)

    constraint_constant_vector_array = read_constraint_constant_vector()
    
    constraint_constant_vector = tf.constant(value = constraint_constant_vector_array, dtype = tf.int64)

    constraint_coefficient_matrix_array = read_constraint_coefficient_matrix()
    
    constraint_coefficient_matrix = tf.constant(value = constraint_coefficient_matrix_array, dtype = tf.int64)
    
    #*******************************************************************************
    #******************************* OPTIMAL VALUES ********************************
    #*******************************************************************************

    # Optimums
    optimal_objective_function_value, optimal_variable_values = create_optimal_objective_function_and_variable_values(maximization_problem, number_of_variables)

    #*******************************************************************************
    #********************** MIXED INTEGER LINEAR PROGRAMMING ***********************
    #*******************************************************************************

    error_code, optimal_objective_function_value, optimal_variable_values = mixed_integer_linear_programming(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix, optimal_objective_function_value, optimal_variable_values)

    condition_problem_feasible = tf.equal(x = error_code, y = 0)
    condition_problem_infeasible = tf.equal(x = error_code, y = 1)
    condition_problem_unbounded = tf.equal(x = error_code, y = 1)
    condition_problem_error = tf.logical_not(x = tf.reduce_any(input_tensor = [condition_problem_feasible, condition_problem_infeasible, condition_problem_unbounded]))
    
    def problem_feasible(optimal_objective_function_value, optimal_variable_values):
        print("\n\n************************************************************************************************************\n")
        print("************************************************************************************************************\n")
        print("*********************************************** FINAL SOLUTION ******************************************\n")
        print("************************************************************************************************************\n")
        print("************************************************************************************************************\n\n\n")

        print_optimal_results(optimal_objective_function_value, optimal_variable_values)
        
        return tf.constant(value = 0, dtype = tf.int64)
    
    def problem_infeasible():
        print("main_function: problem_infeasible: Problem is infeasible!")
        
        return tf.constant(value = 0, dtype = tf.int64)
    
    def problem_unbounded():
        print("main_function: problem_unbounded: Problem is unbounded!")
        
        return tf.constant(value = 0, dtype = tf.int64)
    
    def problem_error(error_code):
        print("main_function: problem_error: How did I get an error_code = {} that is > 2?".format(error_code))
        
        return tf.constant(value = 0, dtype = tf.int64)
    
    def do_nothing():
        return tf.constant(value = 0, dtype = tf.int64)

    problem_error_branch = tf.cond(pred = condition_problem_error, true_fn = lambda: problem_error(error_code), false_fn = lambda: do_nothing())
    problem_unbounded_branch = tf.cond(pred = condition_problem_unbounded, true_fn = lambda: problem_unbounded(), false_fn = lambda: problem_error_branch)
    problem_infeasible_branch = tf.cond(pred = condition_problem_infeasible, true_fn = lambda: problem_infeasible(), false_fn = lambda: problem_unbounded_branch)
    problem_feasible_branch = tf.cond(pred = condition_problem_feasible, true_fn = lambda: problem_feasible(optimal_objective_function_value, optimal_variable_values), false_fn = lambda: problem_infeasible_branch)
    
    _ = problem_feasible_branch

    return error_code, optimal_objective_function_value, optimal_variable_values

# Run algorithm

In [51]:
# For graph execution
with tf.Session() as sess:
    result = sess.run(fetches = main_function())
    
    print("result = \n{}".format(result))

read_objective_function_maximization_or_minimization: maximization_problem = 
1
read_number_of_constraints_and_variables: number_of_constraints = 
4
read_number_of_constraints_and_variables: number_of_variables = 
2
read_and_count_variable_special_requirements: variable_special_requirements = 
[1 1]
read_and_count_variable_special_requirements: number_of_variables_required_to_be_standard = 
0
read_and_count_variable_special_requirements: number_of_variables_required_to_be_integer = 
2
read_and_count_variable_special_requirements: number_of_variables_required_to_be_binary = 
0
read_and_count_variable_special_requirements: number_of_variables_required_to_be_unrestricted = 
0
read_objective_function_initial_constant: objective_function_initial_constant = 
0.0
read_objective_function_coefficient_vector: objective_function_coefficient_vector = 
[ 1. -1.]
read_constraint_inequality_direction_vector: constraint_inequality_direction_vector = 
[-1  1 -1  1]
read_constraint_constant_vector: cons

In [52]:
# For eager execution
# main_function()